In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools as it
import json
from collections import Counter
import math
import time
import random
import platform
import copy

In [2]:
# path to folder where all the files are stored
folder_path = 'C:/Users/varunb/Dropbox/IR_Lab/Asymmetry_Computational/Asymmetry_elegans/Final'

In [3]:
''' small functions used throughout the code'''
def return_contra(neuron):
    ''' for any string ending in L/R,
    a new string ending with R/L is returned '''
    if neuron not in ['PQR','PVR','AVL','RIR','AQR']:
        if neuron.endswith('L'):
            return(neuron[:-1]+'R')
        elif neuron.endswith('R'):
            return(neuron[:-1]+'L')
        else:
            return(neuron)
    else:
        return(neuron)
rc = return_contra
def coinToss():
        flip = random.choice([0,1])
        if flip == 0:
            return('winner')
        elif flip == 1:
            return('loser')
def class_info(sex=str):
    ''' Generates list of bilateral neurons from list of neurons, 
    and gives their class count
    output: LR neurons, class count, LR pairs (AL-R), LR classes (A,B...)
    sex = 'herm' or 'male' 
    common = None or 'common'

    returns:
        neuron: class dic for all neurons
        class: neuron count dic
        bilateral neuron: class dic for only neurons that are bilateral
        bilateral class: neuron count dic
        common neurons: class dic
    '''
    infodf = pd.read_csv(folder_path+'/input_files/celegans_allneurons_info.csv')
    
    bicladf = infodf[infodf['type']== 'bilateral']
    bicladf = bicladf[bicladf['sex'].str.contains(sex)]

    df = infodf[infodf['sex'].str.contains(sex)]
    neucla = {neu:cla for neu,cla in zip(list(df['neuron']),list(df['class']))}
    clacount = dict(Counter(df['class']))
    clacount = {k:v for k,v in clacount.items() if not v <= 1}
 


    bicla = {neu:cla for neu,cla in zip(list(bicladf['neuron']),list(bicladf['class']))}
    biclacount = dict(Counter(bicladf['class']))
    return(neucla,clacount,bicla,biclacount)

In [4]:
''' calculate Fu for directed connectomes'''



def get_directed_asymmetry(filetouse,sourcecol,targetcol,synapsetypecol,dataset=None,common=False,sex='herm' or 'male'):
    
    data = pd.read_csv(filetouse,sep=',')
    common_neurons = pd.read_csv(folder_path+'/input_files/common_neurons_AB01_AB02_SE00.csv',sep = ',',names = ['node'])
    common_neurons['node'] = common_neurons['node'].str.strip("'")

    LR_class_dic = class_info(sex)[2]
    LR_class_count = class_info(sex)[3]

    def return_class(tup=tuple or str):
        '''takes synapse tuple (neuron,neuron) or neuron as input and 
        gives class tuple or neuron class as output'''
        if isinstance(tup, tuple):
            clalis = [LR_class_dic[i] for i in tup if i in LR_class_dic.keys()]
            return(tuple(clalis))
        if isinstance(tup,str):
            if tup in LR_class_dic.keys():
                return(LR_class_dic[tup])
    
    rct = return_class

    regex_pattern = 'BWM|um|anal|int|sph|vm|mc|hyp|pm|intestine|intL|GLRDL|GLRDR|GLRL|GLRR|GLRLR|GLRVL|GLRVR|excgl|sh'
    if common == True:
        data = data.loc[data[sourcecol].isin(common_neurons['node'])]
        data = data.loc[data[targetcol].isin(common_neurons['node'])]
   

    data_chem = data.loc[data[synapsetypecol] == 'chemical']
    data_chem = data_chem[~data_chem[sourcecol].str.contains(regex_pattern,regex=True)]
    data_chem = data_chem[~data_chem[targetcol].str.contains(regex_pattern,regex=True)]
    data_chem.drop_duplicates(inplace=True)
    data_chem.reset_index(inplace=True)

    chem_net = nx.from_pandas_edgelist(data_chem,source=sourcecol,target=targetcol,create_using=nx.DiGraph)
    
    chem_net.remove_edges_from(list(nx.selfloop_edges(chem_net)))
    chem_deg = ["%s %s" % x for x in chem_net.degree()]
    dic_chem_deg = dict(s.split(' ') for s in chem_deg)

    chem_nodelist = nx.nodes(chem_net)
    chem_all_names={}; chem_all_count={}; chem_out_names={};\
         chem_out_count={}; chem_in_names={}; chem_in_count ={}
    for node in chem_nodelist:
        chem_all_names[node] = list(nx.all_neighbors(chem_net, node)); chem_all_count[node] = len(chem_all_names[node])
        chem_out_names[node] = list(chem_net.successors(node)); chem_out_count[node] = len(chem_out_names[node])
        chem_in_names[node] = list(chem_net.predecessors(node)); chem_in_count[node] = len(chem_in_names[node])

    
    def create_contralateral_dict(nodelist, dic):
        '''this function will count the unilateral asymmetric
        connections between the Left and Right pairs of neurons'''
        unique_partners = {}
        L_unique_contlat = {}
        R_unique_contlat = {}
        L_unique_contlat_names = {}
        R_unique_contlat_names = {}
        notLR= ['PQR','PVR','AVL','RIR','AQR']
        for node1, node2 in it.permutations(nodelist,2):
            if (node1.endswith(('R')) and node2.endswith(('L'))):
                if node1[:-1] == node2[:-1]:
                    R_lis = dic[node1]
                    L_lis = dic[node2]
                    def separate_LR(lis):
                        lis_LR = []
                        lis_notLR = []
                        for i in range(len(lis)):
                            if (lis[i] in notLR) or (lis[i].endswith(('L','R')) == False):
                                lis_notLR.append(lis[i])
                            elif lis[i].endswith(('L','R')):
                                lis_LR.append(lis[i])
                        return(lis_notLR,lis_LR)

                    R_lis_notLR, R_lis_LR = separate_LR(R_lis)
                    L_lis_notLR, L_lis_LR = separate_LR(L_lis)

                    def count_asymmetry(L_lis, R_lis, L_lis_notLR,R_lis_notLR,lislen1,lislen2):

                        count1 = 0
                        while count1 < lislen1:
                            [(L_lis_notLR.remove(elem),R_lis_notLR.remove(elem) )for elem in L_lis_notLR if elem in R_lis_notLR]

                            count1 += 1

                        count2 = 0
                        while count2 < lislen2:
                            for elem in L_lis:
                                if elem.endswith(('L')):
                                    if (elem[:-1] + 'R') in R_lis:
                                        L_lis.remove(elem)
                                        R_lis.remove(elem[:-1]+'R')
                                elif elem.endswith(('R')):
                                    if (elem[:-1]+'L') in R_lis:
                                        L_lis.remove(elem)
                                        R_lis.remove(elem[:-1]+'L')
                            count2 += 1

                        L_names = L_lis + list(L_lis_notLR)
                        R_names = R_lis + list(R_lis_notLR)
                        return(L_names,R_names)
                    
                    L_contlat_names, R_contlat_names = count_asymmetry(L_lis_LR, R_lis_LR, L_lis_notLR, R_lis_notLR,len(L_lis_notLR+R_lis_notLR),len(L_lis_LR+R_lis_LR))
                    
                    unique_partners[node1[:-1]+'L'] = L_contlat_names
                    unique_partners[node1[:-1]+'R'] = R_contlat_names
                    L_unique_contlat_names[node2[:-1]+'L-R'] = L_contlat_names
                    R_unique_contlat_names[node1[:-1]+'L-R'] = R_contlat_names
                    L_unique_contlat[node2[:-1]+'L-R'] = len(L_contlat_names)
                    R_unique_contlat[node1[:-1]+'L-R'] = len(R_contlat_names)

        return(L_unique_contlat,R_unique_contlat,L_unique_contlat_names,R_unique_contlat_names,unique_partners)

    L_contlat_chem_in, R_contlat_chem_in, L_contlat_chem_in_names, R_contlat_chem_in_names, unique_partners_in = create_contralateral_dict(chem_nodelist, chem_in_names)
    L_contlat_chem_out, R_contlat_chem_out, L_contlat_chem_out_names, R_contlat_chem_out_names, unique_partners_out = create_contralateral_dict(chem_nodelist, chem_out_names)


    df_unique_partners_in = pd.DataFrame.from_dict(unique_partners_in, orient='index').transpose()
    df_unique_partners_in = df_unique_partners_in.melt(var_name='Target', value_name='Source').dropna()
    df_unique_partners_in['unique_syn'] = (df_unique_partners_in['Source']+'-'+df_unique_partners_in['Target'])

    df_unique_partners_out = pd.DataFrame.from_dict(unique_partners_out, orient='index').transpose()
    df_unique_partners_out = df_unique_partners_out.melt(var_name='Source', value_name='Target').dropna()
    df_unique_partners_out['unique_syn'] = (df_unique_partners_out['Source']+'-'+df_unique_partners_out['Target'])

    unique_partners_df = pd.concat([df_unique_partners_out,df_unique_partners_in], axis=0).sort_values(by='unique_syn')

    LR_pairs = list(set(R_contlat_chem_in.keys()).union(set(R_contlat_chem_out.keys())))
    chem_ne_dic = {}
    for key, val in chem_all_names.items():
        chem_ne_dic[key[:-1]+'L-R'] = val
        
    chem_ne_dic = {key:chem_ne_dic[key] for key in LR_pairs}

    asym_conn_dic = {}
    asym_conn_dic_in = {}
    asym_conn_dic_out = {}
    LR_nodelist = list(set(list(L_contlat_chem_in_names.keys())).union(set(list(L_contlat_chem_out_names.keys())),\
        set(list(R_contlat_chem_in_names.keys())),set(list(R_contlat_chem_out_names.keys()))))
    for node in LR_nodelist:
        asym_conn_dic[node] = L_contlat_chem_in_names[node] + L_contlat_chem_out_names[node] + \
            R_contlat_chem_in_names[node] + R_contlat_chem_out_names[node] 
        asym_conn_dic_in[node] = L_contlat_chem_in_names[node] + R_contlat_chem_in_names[node]
        asym_conn_dic_out[node] = L_contlat_chem_out_names[node] + R_contlat_chem_out_names[node] 

    LR_nodepairs = list(set(L_contlat_chem_in.keys()).union(set(L_contlat_chem_out.keys()),set(R_contlat_chem_in.keys()),set(R_contlat_chem_out.keys())))
    bilat_nodes = []
    for nodepair in LR_nodepairs:
        nodeL, nodeR = nodepair[:-3] + "L" , nodepair[:-3] + "R"
        bilat_nodes.extend((nodeL, nodeR))

    lod = [L_contlat_chem_in,L_contlat_chem_out,R_contlat_chem_in,R_contlat_chem_out] 
    lod_str = ['L_chem_in_unilat', 'L_chem_out_unilat','R_chem_in_unilat','R_chem_out_unilat']

    df_LR_unique_conn = pd.DataFrame(index = sorted(LR_nodepairs))
    df_LR_unique_conn['node'] = df_LR_unique_conn.index
    for i in range(len(lod)):
        df_LR_unique_conn[lod_str[i]] = df_LR_unique_conn['node'].map(lod[i])

    df_LR_unique_conn.reset_index(drop=True,inplace=True) 
    df_LR_unique_conn.fillna(0,inplace=True)
    df_LR_unique_conn['chem_total_unique'] = df_LR_unique_conn['L_chem_in_unilat']+df_LR_unique_conn['R_chem_in_unilat']+\
        df_LR_unique_conn['L_chem_out_unilat']+df_LR_unique_conn['R_chem_out_unilat']

    df_deg = pd.DataFrame(list(dic_chem_deg.values()),index=list(dic_chem_deg.keys()),columns=['deg'])
    df_deg.reset_index(inplace=True)
    df_deg.rename({'index':'node'},axis=1,inplace = True)

    df_deg['deg'] = df_deg['node'].map(dic_chem_deg).astype(int)
    df_deg['chem_deg_in'] = df_deg['node'].map(chem_in_count)
    df_deg['chem_deg_out'] = df_deg['node'].map(chem_out_count)

    df_deg_L = df_deg.loc[df_deg['node'].str.endswith(('L'))].set_index('node')
    df_deg_R = df_deg.loc[df_deg['node'].str.endswith(('R'))].set_index('node')
    for df in [df_deg_L,df_deg_R]:
        df.index = df.index.str[:-1]
        df.reset_index(inplace =True)
    
    df_asym = df_deg_L.merge(df_deg_R,how='inner',on='node',suffixes=('_L','_R'))
    df_asym['node'] = df_asym['node'].astype(str)+'L-R'
    df_asym['chem_deg_total_L'] = df_asym['chem_deg_in_L']+df_asym['chem_deg_out_L']
    df_asym['chem_deg_total_R'] = df_asym['chem_deg_in_R']+df_asym['chem_deg_out_R']
    df_asym['chem_deg_total'] = df_asym['chem_deg_total_L'] +df_asym['chem_deg_total_R']
    df_asym['chem_deg_in_total'] = df_asym['chem_deg_in_L'] + df_asym['chem_deg_in_R']
    df_asym['chem_deg_out_total'] = df_asym['chem_deg_out_L'] + df_asym['chem_deg_out_R']

    df_asym = df_asym.merge(df_LR_unique_conn,how='inner',on='node')
    for lettertouse in ['L','R']:
        df_asym[lettertouse+'_chem_unilat_total'] = df_asym[lettertouse+'_chem_in_unilat'] + df_asym[lettertouse+'_chem_out_unilat']
    
    df_asym['chem_unilat_total_in'] = df_asym['R_chem_in_unilat'] + df_asym['L_chem_in_unilat']
    df_asym['chem_unilat_total_out'] = df_asym['R_chem_out_unilat'] + df_asym['L_chem_out_unilat']


    df_asym['chem_bias'] = df_asym['deg_L']- df_asym['deg_R']
    df_asym['chem_uni_bias'] = df_asym['L_chem_unilat_total'] - df_asym['R_chem_unilat_total']
    df_asym['chem_deg_all_total'] = df_asym['deg_L'] + df_asym['deg_R']
    df_asym['chem_uni_frac_bias'] = df_asym['chem_bias']/(df_asym['L_chem_unilat_total']+df_asym['R_chem_unilat_total'])
    df_asym['chem_frac_asym'] = df_asym['chem_total_unique']/ df_asym['chem_deg_all_total']

    df_asym['chem_abs_delLR_all'] = (np.sqrt(2.0)/(2.0))*((df_asym['deg_L']-df_asym['deg_R']))
    df_asym = df_asym.sort_values(by=['node'])
    df_asym = df_asym.set_index('node')
    
    fu = np.mean(df_asym['chem_frac_asym'])
    df_asym = df_asym.add_prefix(dataset)

    
    return( df_asym, chem_net,data_chem,unique_partners_df,df_unique_partners_in, df_unique_partners_out,fu)
AB01_chem_asym, ab01_chem_net,data_chem_ab01,unique_edges_ab01,unique_edges_in_ab01 ,unique_edges_out_ab01,AB01_fu = get_directed_asymmetry(folder_path+"/raw_data/witvliet_2020_adult_brain_01.csv",'Source','Target', 'Type', \
    dataset="ab01-",  common=True,sex='herm')
AB02_chem_asym, ab02_chem_net,data_chem_ab02,unique_edges_ab02,unique_edges_in_ab02 ,unique_edges_out_ab02,AB02_fu = get_directed_asymmetry(folder_path+"/raw_data/witvliet_2020_adult_brain_02.csv",'Source','Target', 'Type', \
    dataset="ab02-",  common=True,sex='herm')
SE00_chem_asym, se00_chem_net_whole,data_chem_se00,unique_edges_se00,unique_edges_in_se00 ,unique_edges_out_se00,SE00_fu = get_directed_asymmetry(folder_path+"/raw_data/celegans_herm_nopharynx_emmonslab.csv",'Source','Target','Type', \
    dataset="se00-",  common=False,sex='herm')
SE01_chem_asym, se01_chem_net,data_chem_se01,unique_edges_se01,unique_edges_in_se01 ,unique_edges_out_se01,SE01_fu = get_directed_asymmetry(folder_path+"/raw_data/celegans_herm_nopharynx_emmonslab.csv",'Source','Target', 'Type', \
    dataset="se01-",  common=True,sex='herm')
L101_chem_asym, l101_chem_net,data_chem_l101,unique_edges_l101,unique_edges_in_l101 ,unique_edges_out_l101,L101_fu = get_directed_asymmetry(folder_path+"/raw_data/witvliet_2020_L1_brain_01.csv",'Source','Target', 'Type', \
    dataset="l101-",  common=True,sex='herm')
L201_chem_asym, l201_chem_net,data_chem_l201,unique_edges_l201,unique_edges_in_l201 ,unique_edges_out_l201,L201_fu = get_directed_asymmetry(folder_path+"/raw_data/witvliet_2020_L2_brain.csv",'Source','Target', 'Type', \
    dataset="l201-",  common=True,sex='herm')
L301_chem_asym, l301_chem_net,data_chem_l301,unique_edges_l301,unique_edges_in_l301 ,unique_edges_out_l301,L301_fu = get_directed_asymmetry(folder_path+"/raw_data/witvliet_2020_L3_brain.csv",'Source','Target', 'Type', \
    dataset="l301-",  common=True,sex='herm')

In [8]:
''' calculate Fu for undirected connectomes'''
def get_undirected_asymmetry(filename,dataset,common_neurons_only = False or None):
    dataframe = pd.read_csv(filename ,sep=',')
    if 'Type' in dataframe.columns:
        dataframe = dataframe.loc[(dataframe['Type'] == 'chemical')]
        dataframe.drop(columns=['Type','Weight'], inplace=True)
    dataframe.drop_duplicates(inplace=True)
    
    common_neurons = pd.read_csv(folder_path+'/input_files/common_neurons_AB01_AB02_SE00.csv',sep=',',header = None)

    LRneurons = pd.read_csv(folder_path+'/input_files/LR_nodepairs_all.csv',sep=',',header=None)
    LR_list = []
    for node in LRneurons[0]:
        LR_list.append(node[:-3]+'L')
        LR_list.append(node[:-3]+'R')
    regex_pattern = 'BWM|um|anal|int|sph|vm|mc|hyp|pm|intestine|intL|GLRDL|GLRDR|GLRL|GLRR|GLRLR|GLRVL|GLRVR|excgl|sh'
    dataframe = dataframe[~dataframe['Source'].str.contains(regex_pattern,regex=True)]
    dataframe = dataframe[~dataframe['Target'].str.contains(regex_pattern,regex=True)]
    if common_neurons_only:
        dataframe = dataframe.loc[dataframe['Source'].isin(common_neurons[0])]
        dataframe = dataframe.loc[dataframe['Target'].isin(common_neurons[0])]
    
    chem_net = nx.from_pandas_edgelist(dataframe,source='Source',target='Target',create_using=nx.Graph)
    
    chem_net.remove_edges_from(list(nx.selfloop_edges(chem_net)))
    chem_deg = ["%s %s" % x for x in chem_net.degree()]
    dic_chem_deg = dict(s.split(' ') for s in chem_deg)

    chem_nodelist = nx.nodes(chem_net)
    chem_all_names={}; chem_all_count={}
    for node in chem_nodelist:
        chem_all_names[node] = list(nx.all_neighbors(chem_net, node))
        chem_all_count[node] = len(chem_all_names[node])
    def create_contralateral_dict(nodelist, dic):
        '''this function will count the unilateral asymmetric
        connections between the Left and Right pairs of neurons'''
        unique_partners = {}
        L_unique_contlat = {}
        R_unique_contlat = {}
        L_unique_contlat_names = {}
        R_unique_contlat_names = {}
        for node1, node2 in it.permutations(nodelist,2):
            if (node1.endswith(('R')) and node2.endswith(('L'))):
                if node1[:-1] == node2[:-1]:
                    R_lis = dic[node1]
                    L_lis = dic[node2]
                    def separate_LR(lis):
                        lis_LR = []
                        lis_notLR = []
                        for i in range(len(lis)):
                            if (lis[i] in ['PQR','PVR','AVL','RIR','AQR']) or (lis[i].endswith(('L','R')) == False):
                                lis_notLR.append(lis[i])
                            elif lis[i].endswith(('L','R')):
                                lis_LR.append(lis[i])
                        return(lis_notLR,lis_LR)

                    R_lis_notLR, R_lis_LR = separate_LR(R_lis)
                    L_lis_notLR, L_lis_LR = separate_LR(L_lis)

                    def count_asymmetry(L_lis, R_lis, L_lis_notLR,R_lis_notLR,lislen1,lislen2):

                        count1 = 0
                        while count1 < lislen1:
                            [(L_lis_notLR.remove(elem),R_lis_notLR.remove(elem) )for elem in L_lis_notLR if elem in R_lis_notLR]

                            count1 += 1

                        count2 = 0
                        while count2 < lislen2:
                            for elem in L_lis:
                                if elem.endswith(('L')):
                                    if (rc(elem)) in R_lis:
                                        L_lis.remove(elem); R_lis.remove(rc(elem))                                        
                                elif elem.endswith(('R')):
                                    if (rc(elem)) in R_lis:
                                        L_lis.remove(elem); R_lis.remove(rc(elem))                                        
                            
                            count2 += 1

                        L_names = L_lis + list(L_lis_notLR)
                        R_names = R_lis + list(R_lis_notLR)
                        return(L_names,R_names)
                    
                    L_contlat_names, R_contlat_names = count_asymmetry(L_lis_LR, R_lis_LR, L_lis_notLR, R_lis_notLR,len(L_lis_notLR+R_lis_notLR),len(L_lis_LR+R_lis_LR))
                    
                    for n in [node1,node2]:
                        if n.endswith('L'):
                            unique_partners[n] = L_contlat_names
                        elif n.endswith('R'):
                            unique_partners[n] = R_contlat_names
                    unique_partners[node1[:-1]+'L'] = L_contlat_names
                    unique_partners[node1[:-1]+'R'] = R_contlat_names
                    L_unique_contlat_names[node2[:-1]+'L-R'] = L_contlat_names
                    R_unique_contlat_names[node1[:-1]+'L-R'] = R_contlat_names
                    L_unique_contlat[node2[:-1]+'L-R'] = len(L_contlat_names)
                    R_unique_contlat[node1[:-1]+'L-R'] = len(R_contlat_names)

        return(L_unique_contlat,R_unique_contlat,L_unique_contlat_names,R_unique_contlat_names,unique_partners)

    L_contlat, R_contlat, L_contlat_names, R_contlat_names, unique_partners = create_contralateral_dict(chem_nodelist, chem_all_names)

    LR_pairs = list(set(R_contlat.keys()).union(set(L_contlat.keys())))
    chem_ne_dic = {key[:-1]+'L-R': val for key, val in chem_all_names.items()}
        
    unique_synapses =  [neuron[:-3] + side + '-' + partner for dic, side in zip([L_contlat_names, R_contlat_names], ['L', 'R']) for neuron, partners in dic.items() for partner in partners]

    chem_ne_dic = {key:chem_ne_dic[key] for key in LR_pairs}
    asym_conn_dic = {}
    LR_nodelist = list(set(list(L_contlat_names.keys())).union(set(list(R_contlat_names.keys()))))

    for node in LR_nodelist:
        asym_conn_dic[node] = L_contlat_names[node] + R_contlat_names[node]

    LR_nodepairs = LR_pairs

    lod = [L_contlat, R_contlat] 
    lod_str = ['L_unilat','R_unilat']

    df_LR_unique_conn = pd.DataFrame(index = sorted(LR_nodepairs))
    df_LR_unique_conn['node'] = df_LR_unique_conn.index
    for i in range(len(lod)):
        df_LR_unique_conn[lod_str[i]] = df_LR_unique_conn['node'].map(lod[i])
    df_LR_unique_conn.reset_index(drop=True,inplace=True) 
    df_LR_unique_conn.fillna(0,inplace=True)
    df_LR_unique_conn['total_unique'] = df_LR_unique_conn['L_unilat']+df_LR_unique_conn['R_unilat']
    df_deg = pd.DataFrame(list(dic_chem_deg.values()),index=list(dic_chem_deg.keys()),columns=['deg'])
    df_deg.reset_index(inplace=True)
    df_deg.rename({'index':'node'},axis=1,inplace = True)
    df_deg['deg'] = df_deg['node'].map(dic_chem_deg).astype(int)
    df_deg_L = df_deg.loc[df_deg['node'].str.endswith(('L'))].set_index('node')
    df_deg_R = df_deg.loc[df_deg['node'].str.endswith(('R'))].set_index('node')
    for df in [df_deg_L,df_deg_R]:
        df.index = df.index.str[:-1]
        df.reset_index(inplace =True)
    df_asym = df_deg_L.merge(df_deg_R,how='inner',on='node',suffixes=('_L','_R'))
    df_asym['node'] = df_asym['node'].astype(str)+'L-R'
    df_asym = df_asym.merge(df_LR_unique_conn,how='inner',on='node')
    df_asym['bias'] = df_asym['deg_L']- df_asym['deg_R']
    df_asym['uni_bias'] = df_asym['L_unilat'] - df_asym['R_unilat']
    df_asym['deg_total'] = df_asym['deg_L'] + df_asym['deg_R']
    df_asym['uni_frac_bias'] = df_asym['bias']/(df_asym['L_unilat']+df_asym['R_unilat'])
    df_asym['frac_asym'] = round(df_asym['total_unique']/ df_asym['deg_total'],3)
    df_asym['delLR'] = (np.sqrt(2.0)/(2.0))*((df_asym['deg_L']-df_asym['deg_R']))
    df_asym = df_asym.sort_values(by=['node'])
    df_asym = df_asym.set_index('node')
    df_asym = df_asym.add_prefix(dataset)
    

    return(df_asym,chem_net,unique_synapses)

ab01_undi_frac_asym, ab01_undi_net, ab01_undi_unqiue_synapses = get_undirected_asymmetry(folder_path+"/raw_data/witvliet_2020_adult_brain_01.csv",'ab01-',common_neurons_only=True)
ab02_undi_frac_asym, ab02_undi_net, ab02_undi_unqiue_synapses = get_undirected_asymmetry(folder_path+"/raw_data/witvliet_2020_adult_brain_02.csv",'ab02-',common_neurons_only=True)
se01_undi_frac_asym, se01_undi_net, se01_undi_unqiue_synapses = get_undirected_asymmetry(folder_path+"/raw_data/celegans_herm_nopharynx_emmonslab.csv",'se01-',common_neurons_only=True)
ab12_undi_frac_asym, ab12_undi_net, ab12_undi_unqiue_synapses = get_undirected_asymmetry(folder_path+"/raw_data/witvliet_dataset8_vol_contactome.csv",'ab12-',common_neurons_only=True)
se11_undi_frac_asym, se11_undi_net, se11_undi_unqiue_synapses = get_undirected_asymmetry(folder_path+"/raw_data/cel_n2u_nr_adj_contactome.csv",'se11-',common_neurons_only=True)
se00_undi_frac_asym, se00_undi_net, se00_undi_unqiue_synapses = get_undirected_asymmetry(folder_path+"/raw_data/celegans_herm_nopharynx_emmonslab.csv",'se00-',common_neurons_only=False)
l101_undi_frac_asym, l101_undi_net, l101_undi_unqiue_synapses = get_undirected_asymmetry(folder_path+"/raw_data/witvliet_2020_L1_brain_01.csv",'ab01-',common_neurons_only=True)
l201_undi_frac_asym, l201_undi_net, l201_undi_unqiue_synapses = get_undirected_asymmetry(folder_path+"/raw_data/witvliet_2020_L2_brain.csv",'ab01-',common_neurons_only=True)
l301_undi_frac_asym, l301_undi_net, l301_undi_unqiue_synapses = get_undirected_asymmetry(folder_path+"/raw_data/witvliet_2020_L3_brain.csv",'ab01-',common_neurons_only=True)

In [9]:
''' calculate Fu for undirected connectomes, with only undirected network as input'''
def fractional_asymmetry_undirected(network,colname=None):

    nodelist = nx.nodes(network)
    all_names={node: list(nx.all_neighbors(network,node)) for node in nodelist}
    all_count={node: len(all_names[node]) for node in set(all_names)}
 

    def create_contralateral_dict(nodelist, dic):
        '''this function will count the unilateral asymmetric
        connections between the Left and Right pairs of neurons'''
        unique_partners = {}
        L_unique_contlat = {}
        R_unique_contlat = {}
        L_unique_contlat_names = {}
        R_unique_contlat_names = {}
        for node1, node2 in it.permutations(nodelist,2):
            if (node1.endswith(('R')) and node2.endswith(('L'))):
                if node1[:-1] == node2[:-1]:
                    R_lis = dic[node1]
                    L_lis = dic[node2]
                    def separate_LR(lis):
                        lis_LR = []
                        lis_notLR = []
                        for i in range(len(lis)):
                            if (lis[i] in ['PQR','PVR','AVL','RIR','AQR']) or (lis[i].endswith(('L','R')) == False):
                                lis_notLR.append(lis[i])
                            elif lis[i].endswith(('L','R')):
                                lis_LR.append(lis[i])
                        return(lis_notLR,lis_LR)

                    R_lis_notLR, R_lis_LR = separate_LR(R_lis)
                    L_lis_notLR, L_lis_LR = separate_LR(L_lis)

                    def count_asymmetry(L_lis, R_lis, L_lis_notLR,R_lis_notLR,lislen1,lislen2):

                        count1 = 0
                        while count1 < lislen1:
                            [(L_lis_notLR.remove(elem),R_lis_notLR.remove(elem) )for elem in L_lis_notLR if elem in R_lis_notLR]

                            count1 += 1

                        count2 = 0
                        while count2 < lislen2:
                            for elem in L_lis:
                                if elem.endswith(('L')):
                                    if (rc(elem)) in R_lis:
                                        L_lis.remove(elem); R_lis.remove(rc(elem))                                        
                                elif elem.endswith(('R')):
                                    if (rc(elem)) in R_lis:
                                        L_lis.remove(elem); R_lis.remove(rc(elem))                                        
                            
                            count2 += 1

                        L_names = L_lis + list(L_lis_notLR)
                        R_names = R_lis + list(R_lis_notLR)
                        return(L_names,R_names)
                    
                    L_contlat_names, R_contlat_names = count_asymmetry(L_lis_LR, R_lis_LR, L_lis_notLR, R_lis_notLR,len(L_lis_notLR+R_lis_notLR),len(L_lis_LR+R_lis_LR))
                    
                    unique_partners[node1[:-1]+'L'] = L_contlat_names
                    unique_partners[node1[:-1]+'R'] = R_contlat_names
                    L_unique_contlat_names[node2[:-1]+'L-R'] = L_contlat_names
                    R_unique_contlat_names[node1[:-1]+'L-R'] = R_contlat_names
                    L_unique_contlat[node2[:-1]+'L-R'] = len(L_contlat_names)
                    R_unique_contlat[node1[:-1]+'L-R'] = len(R_contlat_names)

        return(L_unique_contlat,R_unique_contlat,L_unique_contlat_names,R_unique_contlat_names,unique_partners)

    L_contlat, R_contlat, L_contlat_names, R_contlat_names, unique_partners = create_contralateral_dict(nodelist, all_names)
    frac_asym = {key: (L_contlat.get(key,[])+R_contlat.get(key,[]))/(all_count[key[:-3]+'L']+all_count[key[:-3]+'R']) for key in set(L_contlat)|set(R_contlat)}
    if colname is not None:
        frac_asym_df = pd.DataFrame.from_dict(frac_asym,orient='index',columns=[colname])
    else:
        frac_asym_df = pd.DataFrame.from_dict(frac_asym,orient='index',columns=['frac_asym'])
    frac_asym_df = frac_asym_df.sort_index(axis=0)

    return(np.mean(list(frac_asym.values())),frac_asym_df)


In [10]:
''' Generate class network from neuron network'''
def network_neuron_classes(network,directed=None,sex='herm' or 'male' ):

    if directed is None:
        print('Error: Specify whether the network is directed or not, directed accepts True or False')
        return
    else:

        dataframe = nx.to_pandas_edgelist(network,source='Source',target='Target')
        regex_pattern = 'BWM|um|anal|int|sph|vm|mc|hyp|pm|intestine|intL|GLRDL|GLRDR|GLRL|GLRR|GLRLR|GLRVL|GLRVR|excgl|sh'
        dataframe = dataframe[~dataframe['Source'].str.contains(regex_pattern,regex=True)]
        dataframe = dataframe[~dataframe['Target'].str.contains(regex_pattern,regex=True)]
        # if common == True:
        #     neuron_class = class_info(sex)[4].values()
        #     neuron_class_dict = class_info(sex)[4]
        #     dataframe = dataframe.loc[dataframe['Source'].str[:3].isin(neuron_class)]
        #     dataframe = dataframe.loc[dataframe['Target'].str[:3].isin(neuron_class)]
        #     dataframe = dataframe.replace(neuron_class_dict)
        # else:
        neuron_class_dict = class_info(sex)[2]
        dataframe = dataframe.replace(neuron_class_dict)

        if directed == False:
            class_net = nx.from_pandas_edgelist(dataframe,source='Source',target='Target',create_using=nx.Graph)
        elif directed == True:
            class_net = nx.from_pandas_edgelist(dataframe,source='Source',target='Target',create_using=nx.DiGraph)
        class_net.remove_edges_from(list(nx.selfloop_edges(class_net)))
        return(class_net)


In [11]:
''' convert undirected network into directed network, while preserving its number of nodes and edges'''
def undirected_to_directed_random(G_undirected):
  
    """
    Converts an undirected graph to a directed graph with randomly assigned directions.

    Args:
    G_undirected: A networkx undirected graph object.

    Returns:
    G_directed: A networkx directed graph object with the same number of edges as the
    undirected graph but with randomly assigned directions.
    """
    G_directed = nx.DiGraph()
    G_directed.add_nodes_from(G_undirected.nodes)

    # Add edges with random directions
    for edge in G_undirected.edges:
        u, v = edge
        direction = random.choice([True, False])
        if direction:
            G_directed.add_edge(u, v)
        else:
            G_directed.add_edge(v, u)
    for edge in G_undirected.edges:
        u, v = edge
        if u not in G_directed.neighbors(v) and v not in G_directed.neighbors(u):
            raise ValueError("Edge conversion failed")

    return(G_directed)

In [12]:
''' calculate Fu for undirected connectomes, with only directed network as input'''
def fractional_asymmetry_directed(network,colname=None):
    ''' given a toy network of LR nodes, 
    calculate the fraction of unique synapses'''
    LR_neuron_pairs = pd.read_csv(folder_path+'/input_files/celegans_LR_neurons_pairs_combined.csv',sep=',',header=None)
    LR_pairs = []
    for pair in LR_neuron_pairs[0]:
        LR_pairs.append(pair[:-3]+'L');LR_pairs.append(pair[:-3]+'R')
    
    def is_bilateral(neuron):
        if neuron in LR_pairs:
            return(True)

    net = network
    net.remove_edges_from(nx.selfloop_edges((net)))
    
    all_edges = list(set(net.edges()))
    not_bilateral_edge = [edge for edge in all_edges if not (is_bilateral(edge[0]) or is_bilateral(edge[1]))]
    asymmetric_synapses = [edge for edge in all_edges \
                           if ((rc(edge[0]),rc(edge[1])) not in all_edges) and (edge != (rc(edge[0]),rc(edge[1]))) and (edge not in not_bilateral_edge)]
    symmetric_synapses = [edge for edge in all_edges \
                          if ((rc(edge[0]),rc(edge[1])) in all_edges) and (edge != rc(edge[0]),rc(edge[1])) and (edge not in not_bilateral_edge)]

    all_neurons = []
    for edg in all_edges:
        all_neurons.append(edg[0]);all_neurons.append(edg[1])

    all_neurons = list(set(all_neurons))
    # all_classes = list(set([neu[:-1] for neu in all_neurons]))
    asymmetry = {neu:[] for neu in all_neurons}

    for edge in asymmetric_synapses:
        #for out-edges
        if is_bilateral(edge[0]):
            if edge[0] not in asymmetry.keys():
                asymmetry.update({edge[0]:[]})
                asymmetry[edge[0]].append(edge[1])
            elif edge[0] in asymmetry.keys():
                asymmetry[edge[0]].append(edge[1])
 
        if is_bilateral(edge[1]):
            if edge[1] not in asymmetry.keys():
                asymmetry.update({edge[1]:[]})
                asymmetry[edge[1]].append(edge[0])
            elif edge[1] in asymmetry.keys():
                asymmetry[edge[1]].append(edge[0])
    asymmetryx = {k: len(asymmetry[k])/nx.degree(net,k) for k in asymmetry.keys()}

    asymmetry = {k1[:-1]+'L-R':(len(asymmetry[k1])+len(asymmetry[k2]))/(nx.degree(net,k1)+nx.degree(net,k2)) for k1,k2 in it.combinations(asymmetry.keys(),2) if k1[:-1]==k2[:-1] and k1!=k2 and is_bilateral(k1)}

    
    if colname is not None:
        asymmetry_df = pd.DataFrame.from_dict(asymmetry, orient='index',columns=[colname])
    else:
        asymmetry_df = pd.DataFrame.from_dict(asymmetry, orient='index')
    fraction_unique_synapses = asymmetry_df[0].mean()
    asymmetry_df = asymmetry_df.sort_index(axis=0)


    return(fraction_unique_synapses,asymmetry_df,asymmetryx)

In [13]:
''' calculate average shortest path length of a Directed network'''
def get_average_shortest_directed_path_length(network):
    '''' directed shortest path will be calculated only for
    the nodes for which such path exists'''
    avg_sp = []
    for n1,n2 in it.permutations(network.nodes(),2):
        if nx.has_path(network,n1,n2):
            avg_sp.append(nx.shortest_path_length(network,n1,n2))
    return(np.mean(avg_sp))


In [14]:
''' DFS algorithm to find all nodes that can be reached with desired path length from a starting node'''
def all_nodes_reachable_dfs(graph, start, length):
    def dfs(node, path_length):
        if path_length == length:
            reachable_nodes.append(node)
            return
        visited.add(node)
        for neighbor in graph[node]:
            if neighbor not in visited:
                dfs(neighbor, path_length + 1)
        visited.remove(node)

    reachable_nodes = []
    visited = set()
    for end in graph:
        if end == start:
            continue
        dfs(start, 0)
    return (list(set(reachable_nodes)))


''' Calculae Redundancy and Reachability for a given network'''
def redundancy_diversity(network,desired_path_length,directed,sex='herm' or 'male'):

    LR_class_dic = class_info(sex)[2]
    LR_class_count = class_info(sex)[3]

    def return_class_tuple(tup=tuple or str):
        '''takes edge or neuron as input and 
        gives class tuple or neuron class as outpu'''
        if isinstance(tup, tuple):
            clalis = [LR_class_dic[i] for i in tup if i in LR_class_dic.keys()]
            return(tuple(clalis))
        if isinstance(tup,str):
            if tup in LR_class_dic.keys():
                return(LR_class_dic[tup])
    
    rct = return_class_tuple

    def neurons_in_class(lis):
        neus = []
        for neu in lis:
            if neu in LR_class_count.keys():
                neus.append(LR_class_count[neu])
        return(math.prod(neus))
    
    # generate list of nodes which are bilateral or belong to class of bilateral neurons
    LRneu = [neu for neu in network.nodes() if neu in LR_class_dic.keys()]

    # generate list of classes in the network
    LR_classes = list(set(rct(tuple(LRneu))))

    # generate dictionary of class pairs for which to find class paths
    if not directed:
        neuron_path_dict = {(cla1,cla2): []  for cla1,cla2 in it.combinations(LR_classes,2) if cla1 != cla2}
    elif directed:
        neuron_path_dict = {(cla1,cla2): []  for cla1,cla2 in it.permutations(LR_classes,2) if cla1 != cla2}

    # find redundant paths between the classes
    for na,nb in it.permutations(LRneu,2):
        if na in network.nodes() and nb in network.nodes():
            if (rct(na) != rct(nb)) and ((rct(na), rct(nb)) in neuron_path_dict.keys()):
                if nx.has_path(network,na,nb):# check if any path between the two nodes
                    # find all simple paths between the nodes and select the paths of desired path-length 
                    paths = [tuple(path) for path in nx.all_simple_paths(network,na,nb,desired_path_length) if len(path) == (desired_path_length+1)]
                    if len(paths)>0:
                        # add the paths to class pair key
                        neuron_path_dict[(rct(na),rct(nb))].append(paths)

    neuron_path_dict = {key:[item for sublist in values for item in sublist] for key,values in neuron_path_dict.items()}

    # count number of paths for each class pair and create dictionary
    red_class_paths = []
    for _,neuron_paths in neuron_path_dict.items():
        class_paths = list(tuple(rct(s) for s in tpl) for tpl in neuron_paths)
        red_class_paths.append(dict(Counter(class_paths)))

    # normalize paths by total realizable paths for each class pair
    red_class_paths_dic = {}
    for d in red_class_paths:
        for key,val in d.items():
            if len(list(set(key))) == len(key):
                red_class_paths_dic[key] = val/neurons_in_class(key)

    # print(sum( red_class_paths_dic.values()),(len(red_class_paths_dic.keys())))
    # convert network into class network
    diversity={cla:[] for cla in LR_classes}
    
    for node in network.nodes():
        if rct(node) in LR_classes:
            # DFS algorithm used to find all classes reached by each class with desired path length
            diversity[rct(node)].extend([rct(reached) for reached in all_nodes_reachable_dfs(network,node,desired_path_length) if rct(reached) in LR_classes and (rct(reached) != rct(node))])
    
    diversity = {k:sorted(list(set(v))) for k,v in diversity.items()}       
    # print(diversity)
    diversity ={key:len((set((val)))) for key,val in diversity.items() if val is not None}

    redundancy = sum( red_class_paths_dic.values())/(len(red_class_paths_dic.keys()))
    reachability = sum(diversity.values())/(len(diversity.keys())*(len(diversity.keys())-1))

    return(round(redundancy,3),round(reachability,3))



In [15]:
''' Calculate Rp1, Su1, FpR1, FuS1'''
def calculate_fuR1fpS1(network, sex='herm' or 'male', directed=bool):
    '''which network takes following arguments:
        hermaphrodite, male, commonSE01AB01AB02, pharynx
        
        returns:fuR1, fpR1, Rp1, fpS1, Sp1, Su1, FuS1'''
    
    neucla = class_info(sex)[2]
    class_counts = class_info(sex)[3]
    class_pairs = {(k1,k2):[] for k1,k2 in it.permutations(list(class_counts.keys()),2) if k1!=k2}

    neu_in_class = {k:[] for k in list(set(neucla.values()))}
    for k,v in neucla.items():
        neu_in_class[v].append(k)

    def neurons_in_class(lis):
        neus = []
        for neu in lis:
            if neu in class_counts.keys():
                neus.append(class_counts[neu])

        return(math.prod(neus))

    def return_class_tuple(tup=tuple or str):
        '''takes edge or neuron as input and 
        gives class tuple or neuron class as output'''
        if isinstance(tup, tuple):
            clalis = [neucla[i] for i in tup if i in neucla.keys()]
            return(tuple(clalis))
        if isinstance(tup,str):
            if tup in neucla.keys():
                return(neucla[tup])

    rct = return_class_tuple

    # find all neuron paths for each class pair
    for k in class_pairs.keys():
        for n1 in neu_in_class[k[0]]:
            for n2 in neu_in_class[k[1]]:
                if network.has_edge(n1,n2):
                    class_pairs[k].append((n1,n2))
    class_pairs = {k:v for k,v in class_pairs.items() if len(v) !=0}

    # for each class pair, find maximum number of neuron paths
    realizable_paths = {k:neurons_in_class(list(k)) for k in class_pairs.keys()}

    paired_synapse_per_class = {k:[] for k in class_pairs.keys()}
    unpaired_synpse_per_class = {k:[] for k in class_pairs.keys()}
    # find paired and unpaired synapses for each class pair
    for k,lis in class_pairs.items():
        for edg in lis:
            rcedg = (rc(edg[0]),rc(edg[1]))
            if rcedg not in lis:
                unpaired_synpse_per_class[k].append(edg)
            elif rcedg in lis:
                paired_synapse_per_class[k].extend([edg,rcedg])

    unpaired_synapse_per_class = {k:v for k,v in unpaired_synpse_per_class.items()}
    paired_synapse_per_class = {k:list(set(v)) for k,v in paired_synapse_per_class.items()}

    avg_unp_syn = {k:len(v)/len(class_pairs[k]) for k,v in unpaired_synapse_per_class.items()}
    avg_pai_syn_prime = {k:len(v)/realizable_paths[k] for k,v in paired_synapse_per_class.items()}
    avg_unp_syn_prime = {k:len(v)/realizable_paths[k] for k,v in unpaired_synapse_per_class.items()}

    r1,s1 = redundancy_diversity(network,1,directed=directed,sex=sex)

    fuR1 = round(sum(avg_unp_syn_prime.values())/len(avg_unp_syn_prime.keys()),3)/r1
    fpR1 = round(sum(avg_pai_syn_prime.values())/len(avg_pai_syn_prime.keys()),3)/r1

    Rp1=sum(avg_pai_syn_prime.values())/len(avg_pai_syn_prime.keys())



    node_class = list(set(rct(tuple(network.nodes()))))


    cla_nodes = {cla:[] for cla in node_class}
    # generate a dictionary for nodes:neighbbors, for neighbors which belong to bilateral class
    node_nei = {n:[i for i in nx.all_neighbors(network,n) if rct(i) is not None] for n in network.nodes() if rct(n) is not None}

    # generate a list of bilateral classes
    LRnodecla = list(set([neu[:-1] for neu in node_nei.keys()]))

    cla_nei = {cla:[] for cla in node_class}
    for node in network.nodes():
        if rct(node) is not None: # check if node belongs to bilateral neurons
            # for each node find all its neighbors which belong to bilateral neurons
            cla_nei[rct(node)].extend([rct(n) for n in nx.all_neighbors(network,node) if rct(n) is not None])
                    
    cla_nei = {k:set(v) for k,v in cla_nei.items()}

    # find all the bilateral neighbors of nodes which can be reached by paired synapses
    cla_paired = {k:[] for k in cla_nodes.keys()}
    cla_unpaired = {k:[] for k in cla_nodes.keys()}
    for cla in LRnodecla:
        if cla+'L' in node_nei.keys() and cla+'R' in node_nei.keys():
            nl = node_nei[cla+'L'];nr = node_nei[cla+'R']
            for neu in nl:
                if rct(neu) != rct(cla+'L'):
                    if rc(neu) in nr:
                        cla_paired[rct(cla+'L')].append(rct(neu))
                    elif rc(neu) not in nr:
                        cla_unpaired[rct(cla+'L')].append(rct(neu))
            for neu in nr:
                if rct(neu) != rct(cla+'L'):
                    if rc(neu) not in nl:
                        cla_unpaired[rct(cla+'L')].append(rct(neu))
                    elif rc(neu) not in nl:
                        cla_unpaired[rct(cla+'L')].append(rct(neu))
            
    cla_paired = {k:sorted(list(set(v))) for k,v in cla_paired.items() if v is not None}
    cla_unpaired = {k:list(set(v)) for k,v in cla_unpaired.items()}
    cla_only_paired = {k:len(list(set(cla_paired[k]).difference(set(cla_unpaired[k])))) for k in cla_paired.keys()}


    only_pai_syn_cla = {k:cla_only_paired[k]/len(cla_nei[k]) for k in cla_only_paired.keys() if len(cla_nei[k]) != 0} #empty dictionary to record the classes reached by paired synapses by each class
    avg_pai_syn_cla = {k:len(v) for k,v in cla_paired.items()}
    avg_unp_syn_cla = {k:len(v) for k,v in cla_unpaired.items()}
    Su1 = sum(avg_unp_syn_cla.values())/((len(avg_unp_syn_cla.keys()))*(len(avg_unp_syn_cla.keys())-1))
    Sp1 = sum(avg_pai_syn_cla.values())/((len(avg_pai_syn_cla.keys()))*(len(avg_pai_syn_cla.keys())-1))


    fpS1 = sum(only_pai_syn_cla.values())/len(only_pai_syn_cla.keys())

    fuS1 = Su1/s1
    # print(round(fuR1prime,3),round(fpR1,3),round(fpS1,3),round(Sp1,3))
    return(round(fuR1,3),round(fpR1,3),round(Rp1,3),round(fpS1,3),round(Sp1,3),round(Su1,3),round(fuS1,3))

In [16]:
''' drive network towards complete asymmetry and symmetry '''

def symmetrize_asymmetrize_network(network_to_asym,network_to_sym,seed,network_size=None,keys_to_use=None,sex='herm' or 'male'):

    print('starting fu', fractional_asymmetry_undirected(network_to_asym)[0])
    random.seed(seed)

    edges_add = list(set((u,v )for u,v in it.permutations(network_to_asym.nodes(),2) if not network_to_asym.has_edge(u,v)))
    
    if keys_to_use == None:
        keys = [str(n/10) for n in range(11)]

    else:
        keys = keys_to_use
    dic ={key:None for key in keys}

    def add_to_dic(dic,net,asymmetry,network_size=network_size):
        fu = asymmetry
        cc = nx.average_clustering(net)
        sp = nx.average_shortest_path_length(net)
        r1,s1 = redundancy_diversity(net,1,False,sex=sex)
        r2,s2 = redundancy_diversity(net,2,False,sex=sex)
        r3,s3 = redundancy_diversity(net,3,False,sex=sex)
        net_to_save = nx.to_dict_of_lists(net)
        if network_size == 'small':
            dic[str(round(asymmetry,ndigits=1))] = {'fu': fu, 'cc': cc, 'sp': sp, 'r1': r1, 'r2': r2, 'r3': r3, 's1': s1, 's2': s2, 's3': s3, 'net': net_to_save, 'seed': seed}
        else:
            dic[str(round(asymmetry,2))] = {'fu': fu, 'cc': cc, 'sp': sp, 'r1': r1, 'r2': r2, 'r3': r3, 's1': s1, 's2': s2, 's3': s3, 'net': net_to_save, 'seed': seed}
        return(dic)
    
    keys_to_check = [float(v) for v in keys]
    print('asymmetrization started')
 
    itr=0
    asymmetry = 0
    # time.sleep(1)
    timeout = time.time() + 30
    while asymmetry < round(max(keys_to_check),ndigits=1) :
        # print(asymmetry)
        edgelist = set((e[0],e[1]) for e in network_to_asym.edges())
        edges_to_add = edges_add
        edges_to_remove = edgelist
        
        edge_to_remove = random.choice(list(edges_to_remove))
        edge_to_add = random.choice(list(edges_to_add))
        # print(edge_to_add,edge_to_remove)
        if not network_to_asym.has_edge(edge_to_add[0],edge_to_add[1]):
            if len(list(network_to_asym.neighbors(edge_to_remove[0]))) >1 and \
                    len(list(network_to_asym.neighbors(edge_to_remove[1]))) >1:
                    network_to_asym.add_edge(edge_to_add[0],edge_to_add[1])
                    network_to_asym.remove_edge(edge_to_remove[0],edge_to_remove[1])
                    if (nx.degree(network_to_asym,edge_to_remove[0])!=0 and nx.degree(network_to_asym,edge_to_remove[1])!=0):
                        asym,asym_df = fractional_asymmetry_undirected(network_to_asym)
                        if (round(asym,4) > round( asymmetry,4)) and (nx.is_connected(network_to_asym)):#this if-else loop acts as a feedback
                            asymmetry = round(asym,4)
                            edges_add.remove(edge_to_add)
                            edges_add.append(edge_to_remove)
                            asymmetry,asym_df = fractional_asymmetry_undirected(network_to_asym)
                            net = network_to_asym.copy()

                            if str(asymmetry) == '1.0':
                                if '1.0' in dic.keys():
                                    dic = add_to_dic(dic,net,asymmetry,network_size)  
                            elif (str(round(asymmetry,2)) in dic.keys()) and (dic[str(round(asymmetry,2))]==None) and (str(round(asymmetry,2))!='1.0'):
                                dic = add_to_dic(dic,net,asymmetry,network_size)
                            elif network_size == 'small':#haven't worked on 0/1 conditions
                                if (str(round(asymmetry,ndigits=1)) in dic.keys()) and (dic[str(round(asymmetry,ndigits=1))]==None) and (str(round(asymmetry,ndigits=1))!='1.0'):
                                    dic = add_to_dic(dic,net,asymmetry,network_size)
                                    # print(dic.keys())
                            itr+=1
                            # print(asymmetry)
                        else:
                            network_to_asym.remove_edge(*edge_to_add)
                            network_to_asym.add_edge(*edge_to_remove)
        elif (network_size == 'small') and (time.time()>timeout):
            print('Timeout')
            break
    print('asymmetrization done')

    print('symmetrization started')
    timeout = time.time() + 30
    sym_itr=0
    symmetry,df = fractional_asymmetry_undirected(network_to_sym)
    sym_edges_add = list(set((u,v )for u,v in it.permutations(network_to_sym.nodes(),2) if not network_to_sym.has_edge(u,v)))
    symmetry = symmetry
    while symmetry> round(min(keys_to_check),ndigits=1):
        sym_edgelist = set((e[0],e[1]) for e in network_to_sym.edges())
        sym_edges_to_add = sym_edges_add
        sym_edges_to_remove = sym_edgelist
        sym_edge_to_remove = random.choice(list(sym_edges_to_remove))
        sym_edge_to_add = random.choice(list(sym_edges_to_add))
        if not network_to_sym.has_edge(sym_edge_to_add[0],sym_edge_to_add[1]):
            if len(list(network_to_sym.neighbors(sym_edge_to_remove[0]))) >1 and \
                    len(list(network_to_sym.neighbors(sym_edge_to_remove[1]))) >1:
                    network_to_sym.add_edge(sym_edge_to_add[0],sym_edge_to_add[1])
                    network_to_sym.remove_edge(sym_edge_to_remove[0],sym_edge_to_remove[1])
                    if (nx.degree(network_to_sym,sym_edge_to_remove[0])!=0) and (nx.degree(network_to_sym,sym_edge_to_remove[1])!=0):
                        asym,asym_df = fractional_asymmetry_undirected(network_to_sym)
                        # print(asymmetry)
                        if (round(asym,4) < round( symmetry,4)) and (nx.is_connected(network_to_sym)):#this if-else loop acts as a feedback
                            symmetry = round(asym,4)
                            # print(symmetry)
                            sym_edges_add.remove(sym_edge_to_add)
                            sym_edges_add.append(sym_edge_to_remove)
                            symmetry,asym_df = fractional_asymmetry_undirected(network_to_sym)
                            net_sym = network_to_sym.copy()
                            if str(symmetry) == '0.0':
                                if '0.0' in dic.keys():
                                    dic = add_to_dic(dic,net_sym,symmetry,network_size)  
                            elif (str(round(symmetry,2)) in dic.keys()) and (dic[str(round(symmetry,2))]==None)and (str(round(symmetry,2))!='0.0'):
                                dic = add_to_dic(dic,net_sym,symmetry,network_size)
                                # print(dic)
                            elif network_size == 'small':
                                if (str(round(symmetry,ndigits=1)) in dic.keys()) and (dic[str(round(symmetry,ndigits=1))]==None) and (str(round(symmetry,ndigits=1))!='0.0'):
                                    dic = add_to_dic(dic,net_sym,symmetry,network_size)
                            sym_itr+=1
                        else:
                            network_to_sym.remove_edge(sym_edge_to_add[0],sym_edge_to_add[1])
                            network_to_sym.add_edge(*sym_edge_to_remove)
    print('symmetrization done')


    return(dic) 



All the codes below generate data for the indicated figures.

In [17]:
''' degree of each neuron, Fig 1A, 1B'''
degdic={}
se01deg={};ab01deg={};ab02deg={}
for net,name,dic in zip([se01_chem_net,ab01_chem_net,ab02_chem_net],['SE01','AB01','AB02'],[se01deg,ab01deg,ab02deg]):
    for tup in nx.degree(net):
        dic[tup[0]] = tup[1]
    degdic[name] = dic

dfdeg = pd.DataFrame.from_dict(degdic,orient='index')
dfdeg = dfdeg.T
dfdeg = dfdeg.sort_index(axis=0)
dfdeg


,SE01,AB01,AB02
ADAL,34,32,28
ADAR,34,29,22
ADEL,41,32,32
ADER,36,38,35
ADFL,30,25,21
...,...,...,...
URXR,48,25,21
URYDL,19,18,19
URYDR,19,13,16
URYVL,23,17,20


In [18]:
''' deg of Left and Right neurons in each class, Fig 1C'''
notlr = ['PQR','PVR','AVL','RIR','AQR']
LRdegdic = {}
se01deg={};ab01deg={};ab02deg={}
for net,name,dic in zip([se01_chem_net,ab01_chem_net,ab02_chem_net],['SE01','AB01','AB02'],[se01deg,ab01deg,ab02deg]):
    for tup in nx.degree(net):
        if tup[0] not in notlr:
            if tup[0].endswith('L'):
                dic[(tup[0][:-1],'Left')] = tup[1]
            elif tup[0].endswith('R'):
                dic[(tup[0][:-1],'Right')] = tup[1]
    LRdegdic[name] = dic
LRdfdeg = pd.DataFrame.from_dict(LRdegdic,orient='index')
LRdfdeg = LRdfdeg.T
LRdfdeg = LRdfdeg.sort_index(axis=0)
LRdfdeg.reset_index(level=1,drop=False,inplace=True)
LRdfdeg = LRdfdeg.pivot(columns=['level_1'])
LRdfdeg

SE01       AB01       AB02      
level_1 Left Right Left Right Left Right
ADA       34    34   32    29   28    22
ADE       41    36   32    38   32    35
ADF       30    37   25    18   21    17
ADL       33    41   27    16   31    22
AFD       15    13   10     8   12    10
...      ...   ...  ...   ...  ...   ...
URAV      20    20   11    11    9     8
URB       19    33   24    20   25    15
URX       41    48   33    25   21    21
URYD      19    19   18    13   19    16
URYV      23    25   17    14   20    22

[83 rows x 6 columns]

In [19]:
'''del dk, eucledian distance of dk from symmetry line, Fig S1A, S1B '''

deldk = pd.DataFrame()
cons = np.sqrt(2)/2

for name in ['SE01','AB01','AB02']:
    deldk[name] = round((LRdfdeg.loc[:,(name,'Left')] - LRdfdeg.loc[:,(name,'Right')])*cons,3)

deldk


,SE01,AB01,AB02
ADA,0.000,2.121,4.243
ADE,3.536,-4.243,-2.121
ADF,-4.950,4.950,2.828
ADL,-5.657,7.778,6.364
AFD,1.414,1.414,1.414
...,...,...,...
URAV,0.000,0.000,0.707
URB,-9.899,2.828,7.071
URX,-4.950,5.657,0.000
URYD,0.000,3.536,2.121


In [20]:
'''absolute del dk, eucledian distance of dk from symmetry line, Fig S1C '''

deldk = pd.DataFrame()
cons = np.sqrt(2)/2

for name in ['SE01','AB01','AB02']:
    deldk[name, 'dk']  = LRdfdeg.loc[:,(name,'Left')] + LRdfdeg.loc[:,(name,'Right')]
    deldk[name,'deldk'] = round((LRdfdeg.loc[:,(name,'Left')] - LRdfdeg.loc[:,(name,'Right')])*cons,3)

deldk

,"(SE01, dk)","(SE01, deldk)","(AB01, dk)","(AB01, deldk)","(AB02, dk)","(AB02, deldk)"
ADA,68,0.000,61,2.121,50,4.243
ADE,77,3.536,70,-4.243,67,-2.121
ADF,67,-4.950,43,4.950,38,2.828
ADL,74,-5.657,43,7.778,53,6.364
AFD,28,1.414,18,1.414,22,1.414
...,...,...,...,...,...,...
URAV,40,0.000,22,0.000,17,0.707
URB,52,-9.899,44,2.828,40,7.071
URX,89,-4.950,58,5.657,42,0.000
URYD,38,0.000,31,3.536,35,2.121


In [21]:
''' Fraction of unpaired synapses, fuk, of each neuron class, Fig 2B, 2C '''

dic={}
for df,name in zip([SE01_chem_asym,AB01_chem_asym,AB02_chem_asym],['se01','ab01','ab02']):
    dic[name] = df[name+'-chem_frac_asym']

fudf = pd.DataFrame.from_dict(dic,orient='index')
fudf = fudf.T
fudf

,se01,ab01,ab02
ADAL-R,0.352941,0.377049,0.320000
ADEL-R,0.402597,0.428571,0.462687
ADFL-R,0.492537,0.395349,0.421053
ADLL-R,0.540541,0.348837,0.320755
AFDL-R,0.428571,0.222222,0.363636
...,...,...,...
URAVL-R,0.400000,0.181818,0.176471
URBL-R,0.500000,0.409091,0.450000
URXL-R,0.483146,0.379310,0.238095
URYDL-R,0.526316,0.290323,0.200000


In [22]:
''' nukL and nukR, the number of unpaired synapses of the left vs. right member of each neuron class, Fig 2D'''

nudic={}
for df,name in zip([SE01_chem_asym,AB01_chem_asym,AB02_chem_asym],['se01','ab01','ab02']):
    nudic[(name,'Left')] = df[name+'-L_chem_unilat_total']
    nudic[(name,'Right')] = df[name+'-R_chem_unilat_total']

nudf = pd.DataFrame.from_dict(nudic,orient='index')
nudf = nudf.T
nudf

se01       ab01       ab02      
        Left Right Left Right Left Right
ADAL-R    12    12   13    10   11     5
ADEL-R    18    13   12    18   14    17
ADFL-R    13    20   12     5   10     6
ADLL-R    16    24   13     2   13     4
AFDL-R     7     5    3     1    5     3
...      ...   ...  ...   ...  ...   ...
URAVL-R    8     8    2     2    2     1
URBL-R     6    20   11     7   14     4
URXL-R    18    25   15     7    5     5
URYDL-R   10    10    7     2    5     2
URYVL-R   11    13    6     3    8    10

[83 rows x 6 columns]

In [23]:
""" Number of unpaired synapses of each individual neuron, nui, and  the neuron degree, di, Fig 3B"""
dicnu={'SE01':{},'AB01':{},'AB02':{}}
for df,name in zip([SE01_chem_asym,AB01_chem_asym,AB02_chem_asym],['se01','ab01','ab02']):
    dicnu[name.upper()].update(dict(zip(df.index.str.replace('L-R','L'),df[name+'-L_chem_unilat_total'])))
    dicnu[name.upper()].update(dict(zip(df.index.str.replace('L-R','R'),df[name+'-R_chem_unilat_total'])))

indnudf = pd.DataFrame.from_dict(dicnu,orient='index')
indnudf = indnudf.T
degnudic ={}
for name in ['SE01','AB01','AB02']:
    degnudic[(name,'nui')] = indnudf[name]
    degnudic[(name,'di')] = dfdeg[name]

degnudf = pd.DataFrame.from_dict(degnudic,orient='index')
degnudf = degnudf.T
degnudf = degnudf.dropna(axis=0)
degnudf = degnudf.sort_index()
degnudf

SE01        AB01        AB02      
        nui    di   nui    di   nui    di
ADAL   12.0  34.0  13.0  32.0  11.0  28.0
ADAR   12.0  34.0  10.0  29.0   5.0  22.0
ADEL   18.0  41.0  12.0  32.0  14.0  32.0
ADER   13.0  36.0  18.0  38.0  17.0  35.0
ADFL   13.0  30.0  12.0  25.0  10.0  21.0
...     ...   ...   ...   ...   ...   ...
URXR   25.0  48.0   7.0  25.0   5.0  21.0
URYDL  10.0  19.0   7.0  18.0   5.0  19.0
URYDR  10.0  19.0   2.0  13.0   2.0  16.0
URYVL  11.0  23.0   6.0  17.0   8.0  20.0
URYVR  13.0  25.0   3.0  14.0  10.0  22.0

[166 rows x 6 columns]

In [24]:
""" Connectome individual neuron fraction of unpaired synapses, fuk, undirected vs. directed, Fig S2 """
fudic={('SE01','undirected'):{},('SE01','directed'):{},('AB01','undirected'):{},('AB01','directed'):{},('AB02','undirected'):{},('AB02','directed'):{}}

for name,dir,und in zip(['se01','ab01','ab02'],[SE01_chem_asym,AB01_chem_asym,AB02_chem_asym],[se01_undi_frac_asym,ab01_undi_frac_asym,ab02_undi_frac_asym]):
    for side in ['L','R']:
        fudic[(name.upper(),'directed')].update(zip(dir.index.str.replace('L-R',side),round(dir[name+'-'+side+'_chem_unilat_total']/dir[name+'-deg_'+side],3)))
        fudic[(name.upper(),'undirected')].update(zip(und.index.str.replace('L-R',side),round(und[name+'-'+side+'_unilat']/und[name+'-deg_'+side],3)))

fudic
fudirund = pd.DataFrame.from_dict(fudic,orient='index')
fudirund = fudirund.T
fudirund

SE01                AB01                AB02         
      undirected directed undirected directed undirected directed
ADAL       0.207    0.353      0.385    0.406      0.370    0.393
ADEL       0.457    0.439      0.321    0.375      0.393    0.438
ADFL       0.435    0.433      0.476    0.480      0.389    0.476
ADLL       0.464    0.485      0.409    0.481      0.360    0.419
AFDL       0.417    0.467      0.375    0.300      0.222    0.417
...          ...      ...        ...      ...        ...      ...
URAVR      0.250    0.400      0.000    0.182      0.125    0.125
URBR       0.538    0.606      0.278    0.350      0.231    0.267
URXR       0.528    0.521      0.227    0.280      0.250    0.238
URYDR      0.471    0.526      0.167    0.154      0.143    0.125
URYVR      0.409    0.520      0.154    0.214      0.476    0.455

[166 rows x 6 columns]

In [25]:
""" Small random networks (40 neurons 80 synapses), fu unidrected vs directed, Fig 4Bi"""

class_count = class_info('herm')[3]
two_neuron_classes = []
for key,val in class_count.items():
    if val == 2:
        two_neuron_classes.append(key)

toy_nodes = []
ex_nodes = []

edges=80
classes = 20
nodes = classes*2
for label in two_neuron_classes[:classes]:
    for side in ('L','R'):
        toy_nodes.append(str(label)+side)
for label in ['A','B','C','D','E','F','G','H','I','J']:
    for side in ('L','R'):
        ex_nodes.append(label+side)
toy_nodes_dict = dict(zip(ex_nodes,toy_nodes))
er_toy4net_dic = {}

''' toy networks'''
seed = 0
while len(er_toy4net_dic)<21*3:
    random.seed(seed)
    er_toy = nx.erdos_renyi_graph(len(toy_nodes),p=0.1)
    if (len(er_toy.nodes()) == nodes) and (len(er_toy.edges()) == edges) and nx.is_connected(er_toy):
        print(seed)
        er_toy = nx.relabel_nodes(er_toy,dict(zip(er_toy.nodes(),toy_nodes)),copy=True)
        ertoy_tosym = er_toy.copy(); ertoy_toasym = er_toy.copy()
        ertoy_undi_symasymed = symmetrize_asymmetrize_network(ertoy_toasym,ertoy_tosym,seed=0,network_size='small')
        for fu in ['0.0','0.3','0.8']:
            ertoy_atfu = nx.from_dict_of_lists(ertoy_undi_symasymed[fu]['net'])
            ertoy_atfu_dir = undirected_to_directed_random(ertoy_atfu)
            er_toy4net_dic[('s'+str(seed),fu,'fu')] = {'undirected':fractional_asymmetry_undirected(ertoy_atfu)[0],'directed':fractional_asymmetry_directed(ertoy_atfu_dir)[0]}

    seed+=1
er_toy4net_df = pd.DataFrame.from_dict(er_toy4net_dic,orient='index')
er_toy4net_df = er_toy4net_df.droplevel(level=2,axis=0)

er_toy4net_df = er_toy4net_df.stack()
er_toy4net_df = er_toy4net_df.swaplevel(1,2)

er_toy4net_df = er_toy4net_df.unstack(0).T
er_toy4net_df = er_toy4net_df.swaplevel(0,1,axis=1)
er_toy4net_df = er_toy4net_df.sort_index(axis=1,level=0,ascending=False)
er_toy4net_df = er_toy4net_df.drop('s23')
er_toy4net_df

23
starting fu 0.8417871017871018
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
149
starting fu 0.8969902319902321
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
180
starting fu 0.8870598845598845
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
218
starting fu 0.8361471861471863
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
226
starting fu 0.9482167832167832
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
227
starting fu 0.9471789321789323
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
240
starting fu 0.8442027417027417
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
359
starting fu 0.8540295815295815
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
379
starting fu 0

0.8                  0.3                  0.0          
     undirected  directed undirected  directed undirected  directed
s149   0.849621  0.922625   0.338224  0.601106        0.0  0.463452
s180   0.849199  1.000000   0.336282  0.718016        0.0  0.623750
s218   0.836854  0.956818   0.344881  0.704365        0.0  0.641250
s226   0.837179  0.900437   0.349849  0.720980        0.0  0.366429
s227   0.849129  0.900443   0.349026  0.803521        0.0  0.538988
s240   0.842417  0.944762   0.347565  0.717615        0.0  0.660952
s359   0.847650  0.976623   0.341022  0.618844        0.0  0.599861
s379   0.840675  0.901627   0.328179  0.687284        0.0  0.604901
s381   0.837540  0.949881   0.348533  0.630048        0.0  0.645806
s396   0.808705  0.923419   0.340408  0.666421        0.0  0.488333
s420   0.825624  0.850949   0.346271  0.816193        0.0  0.641429
s562   0.836378  0.932103   0.349326  0.676344        0.0  0.675476
s597   0.837381  0.893611   0.343462  0.484720        0.0  0.733393
s639   0.842219  0.890750   0.344710  0.695025        0.0  0.538333
s660   0.845009  0.938889   0.336875  0.738910        0.0  0.511270
s686   0.845274  0.941270   0.345387  0.748473        0.0  0.582289
s742   0.833366  0.940714   0.341172  0.743719        0.0  0.656944
s828   0.827882  0.949838   0.342799  0.571769        0.0  0.538790
s908   0.820280  0.953736   0.345288  0.879969        0.0  0.565714
s910   0.849470  0.974242   0.348949  0.815244        0.0  0.577321

In [26]:
""" Small random networks (40 neurons 80 synapses), difference between directed and undirected fu, Fig 4Bii"""

diffdic = {}
for fu in ['0.0','0.3','0.8']:
    diff = er_toy4net_df[(fu,'directed')] - er_toy4net_df[(fu,'undirected')]
    diffdic[fu] = diff

er_toy4net_diff_df = pd.DataFrame.from_dict(diffdic,orient='index')
er_toy4net_diff_df =er_toy4net_diff_df.T
er_toy4net_diff_df

,0.0,0.3,0.8
s149,0.463452,0.262882,0.073004
s180,0.623750,0.381734,0.150801
s218,0.641250,0.359484,0.119964
s226,0.366429,0.371131,0.063258
s227,0.538988,0.454495,0.051313
s240,0.660952,0.370051,0.102345
s359,0.599861,0.277822,0.128973
s379,0.604901,0.359105,0.060952
s381,0.645806,0.281515,0.112341
s396,0.488333,0.326014,0.114714


In [27]:
""" Small random networks (40 neurons 80 synapses), neuron class fraction of unpaired synapses fuk undirected vs. directed networks, Fig 4Biii"""

class_count = class_info('herm')[3]
two_neuron_classes = []
for key,val in class_count.items():
    if val == 2:
        two_neuron_classes.append(key)

toy_nodes = []
ex_nodes = []

edges=80
classes = 20
nodes = classes*2
for label in two_neuron_classes[:classes]:
    for side in ('L','R'):
        toy_nodes.append(str(label)+side)
toy_nodes_dict = dict(zip(list(set([n[:-1]+'L-R' for n in toy_nodes])),[i.upper() for i in list(map(chr, range(97, 117)))]))
toy_nodes_dict

er_toy4net_03_dic = {}

''' toy networks'''
seed = 0
while len(er_toy4net_03_dic)<21*2:
    random.seed(seed)
    er_toy = nx.erdos_renyi_graph(len(toy_nodes),p=0.1)
    if (len(er_toy.nodes()) == nodes) and (len(er_toy.edges()) == edges) and nx.is_connected(er_toy):
        print(seed)
        er_toy = nx.relabel_nodes(er_toy,dict(zip(er_toy.nodes(),toy_nodes)),copy=True)
        ertoy_tosym = er_toy.copy(); ertoy_toasym = er_toy.copy()
        ertoy_undi_symasymed = symmetrize_asymmetrize_network(ertoy_toasym,ertoy_tosym,seed=0,network_size='small')
        ertoy_atfu = nx.from_dict_of_lists(ertoy_undi_symasymed['0.3']['net'])
        er_toy4net_03_dic[('s'+str(seed),'undirected')] = dict(zip(fractional_asymmetry_undirected(ertoy_atfu)[1].index,fractional_asymmetry_undirected(ertoy_atfu)[1]['frac_asym']))
        ertoy_atfu_dir = undirected_to_directed_random(ertoy_atfu)
        er_toy4net_03_dic[('s'+str(seed),'directed')] = dict(zip(fractional_asymmetry_directed(ertoy_atfu_dir)[1].index,fractional_asymmetry_directed(ertoy_atfu_dir)[1][0]))

    seed+=1

er_toy4net_03_dic

er_toy4net_03_df = pd.DataFrame.from_dict(er_toy4net_03_dic,orient='index')
er_toy4net_03_df = er_toy4net_03_df.T
er_toy4net_03_df = er_toy4net_03_df.swaplevel(0,1,axis=1).sort_index(axis=1)
er_toy4net_03_df.rename(toy_nodes_dict,axis=0,inplace=True)
er_toy4net_03_df = er_toy4net_03_df.T
er_toy4net_03_df = er_toy4net_03_df.drop('s23',axis=0,level=1)
er_toy4net_03_df = er_toy4net_03_df.sort_index(axis=1).sort_index(axis=0,ascending=False)
er_toy4net_03_df = round(er_toy4net_03_df,3)
er_toy4net_03_df

23
starting fu 0.8417871017871018
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
149
starting fu 0.8969902319902321
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
180
starting fu 0.8870598845598845
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
218
starting fu 0.8361471861471863
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
226
starting fu 0.9482167832167832
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
227
starting fu 0.9471789321789323
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
240
starting fu 0.8442027417027417
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
359
starting fu 0.8540295815295815
asymmetrization started
asymmetrization done
symmetrization started
symmetrization done
379
starting fu 0

A      B      C      D      E      F      G      H  \
undirected s910  0.714  0.500  0.333  0.556  0.333  0.000  0.000  0.818   
           s908  0.333  0.200  0.692  0.000  0.200  0.556  0.500  0.273   
           s828  0.250  0.000  0.091  0.500  0.800  0.750  0.000  0.529   
           s742  0.333  0.400  0.000  0.500  0.333  0.200  0.571  0.200   
           s686  0.500  0.000  0.667  0.000  0.000  0.429  0.000  0.200   
           s660  0.273  0.556  0.000  0.455  0.538  0.333  0.000  0.833   
           s639  0.200  0.000  0.636  0.556  0.625  0.000  0.600  0.400   
           s597  0.273  0.000  0.000  0.000  0.000  0.750  0.600  0.692   
           s562  0.538  0.333  0.400  0.455  0.000  0.250  0.833  0.333   
           s420  0.692  0.429  0.833  0.000  0.000  0.143  0.200  0.500   
           s396  0.000  0.333  0.600  0.000  0.000  0.000  0.556  0.333   
           s381  0.200  0.333  0.000  0.200  0.692  0.455  0.500  0.000   
           s379  0.000  0.000  0.429  0.333  0.600  0.000  0.636  0.333   
           s359  0.273  0.400  0.818  0.143  0.429  0.200  0.692  0.000   
           s240  0.000  0.200  0.400  0.429  0.500  0.273  0.000  0.000   
           s227  0.467  0.000  0.500  0.000  0.571  0.400  0.600  0.000   
           s226  0.200  0.250  0.500  0.000  0.500  0.400  0.333  0.818   
           s218  0.500  0.333  0.000  0.400  0.333  0.500  0.600  0.600   
           s180  0.600  0.429  0.000  0.000  0.600  0.429  0.400  0.333   
           s149  0.000  0.385  1.000  0.333  0.200  0.429  0.600  0.200   
directed   s910  1.000  1.000  0.667  0.778  1.000  0.000  0.500  1.000   
           s908  0.333  0.200  0.692  0.667  0.800  0.778  1.000  0.636   
           s828  0.500  0.500  0.455  1.000  1.000  1.000  0.000  0.882   
           s742  1.000  0.600  0.000  0.750  0.833  0.600  0.857  0.600   
           s686  0.667  0.500  0.667  0.500  1.000  0.714  1.000  0.333   
           s660  0.636  0.556  0.333  0.818  0.846  0.333  1.000  1.000   
           s639  0.600  1.000  0.818  1.000  0.625  0.333  1.000  0.600   
           s597  0.636  0.500  1.000  0.000  0.000  1.000  0.800  0.846   
           s562  0.692  0.667  0.600  0.818  1.000  0.750  1.000  0.833   
           s420  0.846  1.000  1.000  0.000  0.000  0.429  1.000  0.750   
           s396  0.500  0.556  0.733  0.500  0.500  0.000  0.778  0.667   
           s381  0.400  1.000  1.000  1.000  0.692  0.636  0.833  0.500   
           s379  0.000  1.000  1.000  0.333  0.867  0.000  0.818  1.000   
           s359  0.455  0.800  0.818  0.714  0.714  0.467  0.846  1.000   
           s240  0.500  0.600  0.600  0.714  0.500  0.636  1.000  0.333   
           s227  0.733  0.667  1.000  1.000  0.714  0.800  1.000  0.500   
           s226  0.200  0.750  1.000  0.667  0.500  1.000  0.778  1.000   
           s218  0.500  0.778  1.000  0.600  1.000  0.750  0.600  1.000   
           s180  1.000  0.857  1.000  1.000  0.800  0.714  0.600  0.667   
           s149  1.000  0.538  1.000  1.000  0.200  0.571  0.600  0.200   

                     I      J      K      L      M      N      O      P  \
undirected s910  0.733  0.000  0.385  0.000  0.333  0.000  0.733  0.800   
           s908  0.714  0.333  0.000  0.200  0.000  0.800  0.429  0.692   
           s828  0.000  0.429  0.556  0.429  0.200  0.000  0.500  0.636   
           s742  0.333  0.556  0.000  0.818  0.000  0.333  0.273  0.000   
           s686  0.647  0.333  0.556  0.333  0.500  0.000  0.538  0.571   
           s660  0.250  0.800  0.636  0.000  0.000  0.333  0.429  0.250   
           s639  0.500  0.600  0.200  0.200  0.000  0.091  0.250  0.800   
           s597  0.750  0.500  0.500  0.000  0.200  0.455  0.000  0.538   
           s562  0.750  0.667  0.000  0.200  0.429  0.000  0.636  0.429   
           s420  0.400  0.000  0.429  0.500  0.000  0.500  0.167  1.000   
           s396  0.455  0.500  0.200  0.667  0.143  0.091  0.571  0.800   
           s381  0.200  0.500  0.000  0.667  

In [28]:
""" Connectome fraction of unpaired synpases undirected vs. directed network, Fig 4C """
print('directed')
for df,name, label in zip([SE01_chem_asym,AB01_chem_asym,AB02_chem_asym],['se01','ab01','ab02'],['CeH1c','CeH2c','CeH3c']):
    print(label,np.mean(df[name+'-chem_frac_asym']))
print('undirected')
for df,name, label in zip([se01_undi_frac_asym,ab01_undi_frac_asym,ab02_undi_frac_asym],['se01','ab01','ab02'],['CeH1c','CeH2c','CeH3c']):
    print(label,np.mean(df[name+'-frac_asym']))


directed
CeH1c 0.4322887988702395
CeH2c 0.36900078833333944
CeH3c 0.34429208553878926
undirected
CeH1c 0.3559518072289157
CeH2c 0.3249156626506024
CeH3c 0.3009638554216867


In [29]:
""" Connectome neuron class fraction of unpaired synapses, fuk, undirected vs. directed, Fig 4D"""

dirdfs = [SE01_chem_asym,AB01_chem_asym,AB02_chem_asym]
unddfs = [se01_undi_frac_asym,ab01_undi_frac_asym,ab02_undi_frac_asym]
fuinddic = {}

for dir, und, name, label in zip(dirdfs,unddfs,['se01','ab01','ab02'],['CeH1c','CeH2c','CeH3c']):
    und_asym = dict(zip(und.index,und[name+'-frac_asym']))
    dir_asym = dict(zip(dir.index,round(dir[name+'-chem_frac_asym'],3)))
    fuinddic[(label,'undirected')] = und_asym
    fuinddic[(label,'directed')] = dir_asym

fuinddf = pd.DataFrame.from_dict(fuinddic,orient='index')
fuinddf = fuinddf.T
fuinddf

CeH1c               CeH2c               CeH3c         
        undirected directed undirected directed undirected directed
ADAL-R       0.220    0.353      0.360    0.377      0.292    0.320
ADEL-R       0.387    0.403      0.377    0.429      0.414    0.463
ADFL-R       0.447    0.493      0.405    0.395      0.312    0.421
ADLL-R       0.483    0.541      0.297    0.349      0.273    0.321
AFDL-R       0.364    0.429      0.286    0.222      0.222    0.364
...            ...      ...        ...      ...        ...      ...
URAVL-R      0.226    0.400      0.048    0.182      0.125    0.176
URBL-R       0.442    0.500      0.333    0.409      0.412    0.450
URXL-R       0.493    0.483      0.261    0.379      0.273    0.238
URYDL-R      0.419    0.526      0.259    0.290      0.200    0.200
URYVL-R      0.381    0.500      0.241    0.290      0.421    0.429

[83 rows x 6 columns]

In [30]:
""" Connectome mean shortest path, SP, for neuron network and class network Fig 5D"""

dirnet = [se01_chem_net,ab01_chem_net,ab02_chem_net]
undnet = [se01_undi_net,ab01_undi_net,ab02_undi_net]
names = ['CeH1c','CeH2c','CeH3c']

spdic = {}

for und,dir,name in zip(undnet,dirnet,names):
    claund = network_neuron_classes(und,directed=False,sex='herm')
    cladir = network_neuron_classes(dir,directed=True,sex='herm')
    und = {'individual neuron':nx.average_shortest_path_length(und),'neuron class':nx.average_shortest_path_length(claund)}
    dir = {'individual neuron':get_average_shortest_directed_path_length(dir),'neuron class':get_average_shortest_directed_path_length(cladir)}
    spdic[(name,'undirected')] = und
    spdic[(name,'directed')] = dir

spdf = pd.DataFrame.from_dict(spdic,orient='index')
spdf = spdf.T
spdf

CeH1c                CeH2c                CeH3c  \
                  undirected  directed undirected  directed undirected   
individual neuron   2.071570  2.478243   2.206269  2.881241   2.217753   
neuron class        1.744655  1.970491   1.870220  2.258752   1.886179   

                             
                   directed  
individual neuron  2.841263  
neuron class       2.269102

In [32]:
""" Simple model (example) network, Redundancy Rn, Reachability, Sn, and mean shortest path, SP vs.  fraction of unpaired synapses, fu, Fig 5E, 5F"""

f = open(folder_path+'/input_files/celegans_class_count_commonSE01AB01AB02.json','r')
LR_class_count = json.load(f)

two_neuron_classes = []
for key,val in LR_class_count.items():
    if val == 2:
        two_neuron_classes.append(key)
LR_classes = pd.read_csv(folder_path+'/input_files/celegans_LR_neurons_classes_commonSE01AB01AB02.csv',header=None)
LR_classes = list(LR_classes[0])
toy_nodes = []
ex_nodes = []

classes = 8
nodes = classes*2
edges=80
for label in two_neuron_classes[:classes]:
    for side in ('L','R'):
        toy_nodes.append(str(label)+side)
for label in ['A','B','C','D','E','F','G','H']:
    for side in ('L','R'):
        ex_nodes.append(label+side)
toy_nodes_dict = dict(zip(toy_nodes,ex_nodes))

exsym = nx.read_gml(folder_path+'/network_files/er_example_sym.gml')
exmid = nx.read_gml(folder_path+'/network_files/er_example_half.gml')
exasym = nx.read_gml(folder_path+'/network_files/er_example_asym.gml')

infdic = {}
for net,name in zip([exsym,exmid,exasym],['sym','mid','asym']):
    rs = [fractional_asymmetry_undirected(net)[0]]
    for pl in range(1,4):
        r,s = redundancy_diversity(net,pl,directed=False,sex='herm')
        rs.extend([r,s])
    rs.append(nx.average_shortest_path_length(net))
    infdic[name] = rs
columns=['Fu','R1','R2','R3','S1','S2','S3','SP']
infdf = pd.DataFrame.from_dict(infdic,orient='index',columns=columns)
infdf
# exsym = nx.relabel_nodes(exsym,toy_nodes_dict,copy=True)
# exmid = nx.relabel_nodes(exmid,toy_nodes_dict,copy=True)
# exasym = nx.relabel_nodes(exasym,toy_nodes_dict,copy=True)


# exmid.nodes()

,Fu,R1,R2,R3,S1,S2,S3,SP
sym,0.00000,0.500,0.321,0.250,0.571,0.125,0.536,3.333333
mid,0.41875,0.396,0.429,0.163,0.714,0.077,0.857,2.775000
asym,1.00000,0.317,0.536,0.172,0.571,0.096,0.786,3.308333


In [33]:
"""Connectome Redundancy, R1, and Reachability, S1, vs. fraction of unpaired synapses, fu, Fig 5G"""

undnet = [se01_undi_net,ab01_undi_net,ab02_undi_net]
names = ['CeH1c','CeH2c','CeH3c']

for net,name in zip(undnet,names):
    fu = fractional_asymmetry_undirected(net)[0]
    r1,s1 = redundancy_diversity(net,1,directed=False,sex='herm')
    print(name,fu,r1,s1)

CeH1c 0.3559905915262085 0.407 0.328
CeH2c 0.3249516596414129 0.431 0.266
CeH3c 0.3010322986946989 0.43 0.252


In [34]:
''' Generating SR, ER and DCB network'''

# Generating SR network
ave_edges = np.average([2007,1669,1633])
seed = 1317
sr_graph = nx.gnm_random_graph(180,ave_edges,seed)
sr_graph = nx.relabel_nodes(sr_graph,dict(zip(sr_graph.nodes(),se01_undi_net.nodes())),copy=False)# relabel the SR network nodes with CeH1c node labels

# Generating ER network
er_grpah = nx.erdos_renyi_graph(len(list(se01_undi_net.nodes())),p=0.124,seed=1319)

# Generating the DCB network
ref_deg ={t[0]:t[1] for t in nx.degree(se01_undi_net)}

# LR_list = pd.read_csv('LR_neurons_separate.csv',header=None)
# LR_list = list(LR_list[0])
# LR_list_copy = copy.copy(LR_list)
dcb_net = nx.Graph()
dcb_net.add_nodes_from(se01_undi_net.nodes())

asym_edges = {}
expected_degrees = list(ref_deg.values())
nodelist = list(ref_deg.keys())


random.seed(10)
random.shuffle(nodelist)
i=1
for neu,deg in zip(nodelist,expected_degrees):
    choice_list = list(set([n for n in nx.neighbors(se11_undi_net,neu)]+[n for n in nx.neighbors(se01_undi_net,neu)]))#
    alternate_choice_list = list(nodelist)
    j=0
    while (nx.degree(dcb_net,neu) < deg) and (j < 180):
        if len(choice_list) > 0:
            target = random.choice(choice_list)
            choice_list.remove(target)
        elif len(alternate_choice_list) > 0:
            target = alternate_choice_list[j]
        if target not in dcb_net.nodes():
            dcb_net.add_node(target)
        if (not dcb_net.has_edge(neu,target)) and (neu != target) and (dcb_net.degree(target) < list(expected_degrees)[list(nodelist).index(target)]):
            dcb_net.add_edge(neu,target)
        
        j+=1
    i+=1

"""Artificial random neural networks ( SR,ER,DCB): Redundancy, R1, and Reachability, S1, vs. fraction of unpaired synapses, fu"""

def return_dict_from_json(filename):
    f = open(filename,'r')
    dic = json.load(f)
    f.close()
    return(dic)

srdic = return_dict_from_json(folder_path+'/network_files/sr_rand_inst1_s1317.json')
erdic = return_dict_from_json(folder_path+'/network_files/er_rand_inst1_s1319.json')
dcbdic = return_dict_from_json(folder_path+'/network_files/deg_rand_inst1_s10.json')

dic = {}

for d,name in zip([srdic,erdic,dcbdic],['SR','ER','DCB']):

    for fu in [str(i/10) for i in range(11)]:
        net = nx.from_dict_of_lists(d[fu]['net'])
        r1,s1 = redundancy_diversity(net,1,directed=False,sex='herm')
        rfu = fractional_asymmetry_undirected(net)[0]
        inf = {'Fu':rfu,'R1':r1,'S1':s1}
        dic[(name,fu)] = inf

df = pd.DataFrame.from_dict(dic,orient='columns')
df = df.T
df


Fu     R1     S1
SR  0.0  0.000000  0.378  0.229
    0.1  0.104877  0.358  0.244
    0.2  0.204938  0.332  0.273
    0.3  0.304904  0.311  0.299
    0.4  0.404925  0.293  0.329
    0.5  0.504150  0.276  0.359
    0.6  0.604548  0.263  0.387
    0.7  0.704252  0.252  0.415
    0.8  0.804427  0.242  0.447
    0.9  0.896165  0.234  0.466
    1.0  1.000000  0.225  0.486
ER  0.0  0.000000  0.405  0.266
    0.1  0.104669  0.382  0.288
    0.2  0.204607  0.351  0.314
    0.3  0.304997  0.328  0.339
    0.4  0.404701  0.305  0.371
    0.5  0.504829  0.289  0.406
    0.6  0.604794  0.274  0.437
    0.7  0.704462  0.262  0.464
    0.8  0.804241  0.251  0.496
    0.9  0.895445  0.245  0.514
    1.0  1.000000  0.235  0.541
DCB 0.0  0.000000  0.424  0.260
    0.1  0.104991  0.396  0.282
    0.2  0.204710  0.368  0.314
    0.3  0.304432  0.349  0.338
    0.4  0.404158  0.328  0.367
    0.5  0.504476  0.311  0.391
    0.6  0.604354  0.297  0.419
    0.7  0.704479  0.288  0.437
    0.8  0.795116  0.272  0.464
    0.9  0.895081  0.260  0.486
    1.0  1.000000  0.248  0.517

In [35]:
""" Connectome networks: Redundancy, R1, and Reachability, S1, vs. fraction of unpaired synapses, fu, Fig 6D"""

se01dic = return_dict_from_json(folder_path+"/network_files/se01_undi_symmetrized.json")
ab01dic = return_dict_from_json(folder_path+"/network_files/ab01_undi_symmetrized.json")
ab02dic = return_dict_from_json(folder_path+"/network_files/ab02_undi_symmetrized.json")

dic = {}

for d,name in zip([se01dic,ab01dic,ab02dic],['CeH1c','CeH2c','CeH3c']):

    for fu in [str(i/10) for i in range(11)]:
        net = nx.from_dict_of_lists(d[fu]['net'])
        r1,s1 = redundancy_diversity(net,1,directed=False,sex='herm')
        rfu = fractional_asymmetry_undirected(net)[0]
        inf = {'Fu':rfu,'R1':r1,'S1':s1}
        dic[(fu,name)] = inf
       
df = pd.DataFrame.from_dict(dic,orient='index')
df = df.T
df = df.stack()
df = df.swaplevel(0,1)
df= df.T
df = df.sort_index(axis=1)
df

CeH1c                   CeH2c                   CeH3c              
           Fu     R1     S1        Fu     R1     S1        Fu     R1     S1
0.0  0.000000  0.502  0.238  0.000000  0.503  0.197  0.000000  0.505  0.185
0.1  0.104695  0.469  0.266  0.104882  0.465  0.224  0.104966  0.457  0.217
0.2  0.204890  0.433  0.299  0.204037  0.449  0.245  0.204540  0.433  0.243
0.3  0.304800  0.414  0.318  0.303942  0.430  0.263  0.300947  0.430  0.252
0.4  0.396448  0.394  0.339  0.396159  0.396  0.288  0.395736  0.388  0.278
0.5  0.495969  0.358  0.373  0.496322  0.358  0.316  0.495934  0.353  0.306
0.6  0.595805  0.328  0.406  0.595487  0.333  0.342  0.595997  0.323  0.336
0.7  0.695268  0.303  0.437  0.695279  0.307  0.369  0.695132  0.300  0.367
0.8  0.795726  0.284  0.467  0.795439  0.284  0.391  0.795094  0.279  0.399
0.9  0.895917  0.269  0.497  0.895477  0.269  0.417  0.896008  0.259  0.432
1.0  1.000000  0.255  0.528  1.000000  0.252  0.446  1.000000  0.244  0.461

In [36]:
"""200 instantitions of artificial simple random network (SR) with distinct fraction of unpaired synapses, fu, 6E"""

sr200 = return_dict_from_json(folder_path+'/network_files/sr_toy200net_large.json')

sr200dic = {}

for s,d in sr200.items():
    for fu in ['0.3', '0.6', '0.9']:
        net = nx.from_dict_of_lists(d[fu]['net'])
        rfu = fractional_asymmetry_undirected(net)[0]
        r1,s1 = redundancy_diversity(net,1,directed=False,sex='herm')
        inf = {'Fu':rfu,'R1':r1,'S1':s1}
        sr200dic[(s,fu)] = inf

df = pd.DataFrame.from_dict(sr200dic,orient='index')
df = df.T
df = df.stack()
df = df.swaplevel(0,1)
df= df.T
df = df.sort_index(axis=1)
df

0.3                     0.6                     0.9              
            Fu     R1     S1        Fu     R1     S1        Fu     R1     S1
1429  0.304323  0.318  0.311  0.604944  0.271  0.388  0.895034  0.237  0.463
1430  0.304834  0.314  0.307  0.604938  0.267  0.394  0.895803  0.235  0.476
1431  0.304635  0.313  0.297  0.604757  0.270  0.381  0.895841  0.235  0.471
1432  0.304954  0.320  0.310  0.604892  0.268  0.400  0.896090  0.240  0.473
1433  0.303747  0.305  0.298  0.604318  0.258  0.381  0.895702  0.231  0.463
...        ...    ...    ...       ...    ...    ...       ...    ...    ...
1624  0.304971  0.313  0.306  0.604944  0.264  0.396  0.895985  0.237  0.478
1625  0.304503  0.306  0.305  0.603916  0.262  0.391  0.897751  0.233  0.468
1626  0.303992  0.315  0.303  0.604909  0.264  0.383  0.895521  0.234  0.468
1627  0.304934  0.310  0.292  0.604972  0.263  0.374  0.895784  0.229  0.464
1628  0.304507  0.307  0.310  0.604390  0.262  0.392  0.896077  0.234  0.477

[200 rows x 9 columns]

In [37]:
"""Connectome networks: Redundancy due exclusively to paired synapses, Rp,1, plotted against the fraction of unpaired synapses, fu, Fig 7A"""
se01dic = return_dict_from_json(folder_path+"/network_files/se01_undi_symmetrized.json")
ab01dic = return_dict_from_json(folder_path+"/network_files/ab01_undi_symmetrized.json")
ab02dic = return_dict_from_json(folder_path+"/network_files/ab02_undi_symmetrized.json")

dic = {}

for d,name in zip([se01dic,ab01dic,ab02dic],['CeH1c','CeH2c','CeH3c']):
    for fu in [str(i/10) for i in range(11)]:
        net = nx.from_dict_of_lists(d[fu]['net'])
        r1,s1 = redundancy_diversity(net,1,directed=False,sex='herm')
        rfu = fractional_asymmetry_undirected(net)[0]
        Rp1 = calculate_fuR1fpS1(net,sex='herm',directed=False)[2]
        inf = {'Fu':rfu,'R1':r1,'Rp1':Rp1}
        dic[(fu,name)] = inf
       
df = pd.DataFrame.from_dict(dic,orient='index')
df = df.T
df = df.stack()
df = df.swaplevel(0,1)
df= df.T
df = df.sort_index(axis=1)
df

CeH1c                   CeH2c                   CeH3c              
           Fu     R1    Rp1        Fu     R1    Rp1        Fu     R1    Rp1
0.0  0.000000  0.502  0.502  0.000000  0.503  0.503  0.000000  0.505  0.505
0.1  0.104695  0.469  0.413  0.104882  0.465  0.412  0.104966  0.457  0.398
0.2  0.204890  0.433  0.334  0.204037  0.449  0.346  0.204540  0.433  0.335
0.3  0.304800  0.414  0.287  0.303942  0.430  0.294  0.300947  0.430  0.301
0.4  0.396448  0.394  0.239  0.396159  0.396  0.241  0.395736  0.388  0.240
0.5  0.495969  0.358  0.183  0.496322  0.358  0.186  0.495934  0.353  0.184
0.6  0.595805  0.328  0.136  0.595487  0.333  0.142  0.595997  0.323  0.136
0.7  0.695268  0.303  0.095  0.695279  0.307  0.099  0.695132  0.300  0.095
0.8  0.795726  0.284  0.059  0.795439  0.284  0.057  0.795094  0.279  0.060
0.9  0.895917  0.269  0.029  0.895477  0.269  0.030  0.896008  0.259  0.028
1.0  1.000000  0.255  0.000  1.000000  0.252  0.000  1.000000  0.244  0.000

In [38]:
""" Connectome networks: fraction of Redundancy due only to paired synapses out of total Redundancy , fpR1, Fig 7B"""
undnet = [se01_undi_net,ab01_undi_net,ab02_undi_net]
names = ['CeH1c','CeH2c','CeH3c']
for net, name in zip(undnet,names):
    FpR1 = calculate_fuR1fpS1(net,sex='herm',directed=False)[1]
    print(name,FpR1)

CeH1c 0.644
CeH2c 0.664
CeH3c 0.7


In [39]:
"""Connectome networks: Reachability due exclusively to unpaired synapses, Sp1, plotted against the fraction of unpaired synapses, fu, Fig 7C"""
se01dic = return_dict_from_json(folder_path+"/network_files/se01_undi_symmetrized.json")
ab01dic = return_dict_from_json(folder_path+"/network_files/ab01_undi_symmetrized.json")
ab02dic = return_dict_from_json(folder_path+"/network_files/ab02_undi_symmetrized.json")

dic = {}

for d,name in zip([se01dic,ab01dic,ab02dic],['CeH1c','CeH2c','CeH3c']):
    for fu in [str(i/10) for i in range(11)]:
        net = nx.from_dict_of_lists(d[fu]['net'])
        r1,s1 = redundancy_diversity(net,1,directed=False,sex='herm')
        rfu = fractional_asymmetry_undirected(net)[0]
        Su1 = calculate_fuR1fpS1(net,sex='herm',directed=False)[5]
        inf = {'Fu':rfu,'S1':s1,'Su1':Su1}
        dic[(fu,name)] = inf
       
df = pd.DataFrame.from_dict(dic,orient='index')
df = df.T
df = df.stack()
df = df.swaplevel(0,1)
df= df.T
df = df.sort_index(axis=1)
df

CeH1c                   CeH2c                   CeH3c              
           Fu     S1    Su1        Fu     S1    Su1        Fu     S1    Su1
0.0  0.000000  0.238  0.000  0.000000  0.197  0.000  0.000000  0.185  0.000
0.1  0.104695  0.266  0.065  0.104882  0.224  0.054  0.104966  0.217  0.059
0.2  0.204890  0.299  0.133  0.204037  0.245  0.108  0.204540  0.243  0.111
0.3  0.304800  0.318  0.185  0.303942  0.263  0.152  0.300947  0.252  0.146
0.4  0.396448  0.339  0.234  0.396159  0.288  0.197  0.395736  0.278  0.189
0.5  0.495969  0.373  0.292  0.496322  0.316  0.247  0.495934  0.306  0.240
0.6  0.595805  0.406  0.344  0.595487  0.342  0.285  0.595997  0.336  0.283
0.7  0.695268  0.437  0.393  0.695279  0.369  0.329  0.695132  0.367  0.329
0.8  0.795726  0.467  0.443  0.795439  0.391  0.370  0.795094  0.399  0.373
0.9  0.895917  0.497  0.486  0.895477  0.417  0.406  0.896008  0.432  0.418
1.0  1.000000  0.528  0.528  1.000000  0.446  0.446  1.000000  0.461  0.461

In [40]:
""" Connectome networks: fraction of Redundancy due only to paired synapses out of total Redundancy , fpR1, Fig 7D"""
undnet = [se01_undi_net,ab01_undi_net,ab02_undi_net]
names = ['CeH1c','CeH2c','CeH3c']
for net, name in zip(undnet,names):
    FuS1 = calculate_fuR1fpS1(net,sex='herm',directed=False)[6]

    print(name,FuS1)

CeH1c 0.652
CeH2c 0.611
CeH3c 0.578


In [42]:
""" Series of small random networks (40 neurons) with variable relative degree, fd, Fig 7E"""

# f = open(folder_path+'/Paper_data/inputs/neuron_lists/class_neuron_count/celegans_class_count_commonSE01AB01AB02.json','r')
# LR_class_count = json.load(f)

# two_neuron_classes = []
# for key,val in LR_class_count.items():
#     if val == 2:
#         two_neuron_classes.append(key)


# LR_classes = pd.read_csv(folder_path+'/Paper_data/inputs/neuron_lists/LR_classes/celegans_LR_neurons_classes_commonSE01AB01AB02.csv',header=None)
# LR_classes = list(LR_classes[0])
# toy_nodes = []
# ex_nodes = []

# classes = 20
# nodes = classes*2
# for label in two_neuron_classes[:classes]:
#     for side in ('L','R'):
#         toy_nodes.append(str(label)+side)
# sr_toy4net_dic = {}


# list_of_edges = [40,78,98,156,312,624]
# edges_constructed = []
# for edges in list_of_edges:
#     seed = edges
#     while (edges not in edges_constructed):
#         random.seed(seed)
#         sr_toy = nx.gnm_random_graph(n=nodes,m=edges)
#         if (len(sr_toy.nodes()) == nodes) and (len(sr_toy.edges()) == edges) and nx.is_connected(sr_toy):
#             print(seed,edges)
#             sr_toy = nx.relabel_nodes(sr_toy,dict(zip(sr_toy.nodes(),toy_nodes)),copy=True)
#             sr_toy4net_dic['e'+str(edges)+'_s'+str(seed)] = sr_toy
#             edges_constructed.append(edges)
#         seed+=1

toy4R1={'0.0':{},'0.2':{},'0.4':{},'1.0':{}}
toy4S1={'0.0':{},'0.2':{},'0.4':{},'1.0':{}}

f = open(folder_path+'/network_files/sr_toy4Enet_dic_all.json','r')
sr_toy4net_dic_sym_asymed = json.load(f)
f.close()

for s,dic in sr_toy4net_dic_sym_asymed.items():
    dic = {k:v for k,v in dic.items() if v is not None}
    
    for fu in ['0.0','0.2','0.4','1.0']:
        if fu in dic.keys():
            net = nx.from_dict_of_lists(dic[fu]['net'])
            r1,s1 = redundancy_diversity(net,1,directed=False,sex='herm')
            maxedges = (40*39)/2
            fd = round(len(net.edges())/maxedges,3)
            if fd < 0.25:
                toy4R1[fu].update({fd:r1})
                toy4S1[fu].update({fd:s1})
    
toy4R1df = pd.DataFrame.from_dict(toy4R1,orient='index')
toy4R1df = toy4R1df.T
print(toy4R1df)
toy4S1df = pd.DataFrame.from_dict(toy4S1,orient='index')
toy4S1df = toy4S1df.T
print(toy4S1df)


         0.0    0.2    0.4    1.0
0.051  0.500  0.398  0.346  0.250
0.100  0.531  0.419  0.362  0.287
0.126  0.523  0.425  0.371  0.295
0.200  0.570  0.490  0.415  0.336
         0.0    0.2    0.4    1.0
0.051  0.100  0.116  0.137  0.211
0.100  0.168  0.211  0.258  0.358
0.126  0.226  0.279  0.316  0.437
0.200  0.337  0.405  0.479  0.611


In [43]:
""" Series of small random networks (40 neurons) with variable relative degree, fd, Fig S3"""
f = open(folder_path+'/network_files/sr_toy4Enet_dic_all.json','r')
sr_toy4net_dic_sym_asymed = json.load(f)
f.close()

inf = {}
for s,dic in sr_toy4net_dic_sym_asymed.items():
    dic = {k:v for k,v in dic.items() if v is not None}
    
    for fu in [str(i/10) for i in range(11)]:
        if fu in dic.keys():
            net = nx.from_dict_of_lists(dic[fu]['net'])
            r1,s1 = redundancy_diversity(net,1,directed=False,sex='herm')
            rfu = fractional_asymmetry_undirected(net)[0]
            maxedges = (40*39)/2
            fd = round(len(net.edges())/maxedges,3)
            inf[(fu,fd)] = {'fu':rfu,'R1':r1,'S1':s1}

df = pd.DataFrame.from_dict(inf,orient='index')
df = df.T
df = df.stack()
df = df.swaplevel(0,1)
df= df.T
df = df.sort_index(axis=1)
df

0.051                   0.100                   0.126                   \
        R1     S1        fu     R1     S1        fu     R1     S1        fu   
0.0  0.500  0.100  0.000000  0.531  0.168  0.000000  0.523  0.226  0.000000   
0.1  0.417  0.111  0.144365  0.472  0.189  0.139303  0.506  0.237  0.131389   
0.2  0.398  0.116  0.246032  0.419  0.211  0.243987  0.425  0.279  0.246259   
0.3  0.375  0.126  0.340714  0.375  0.242  0.340812  0.381  0.311  0.348344   
0.4  0.346  0.137  0.420000  0.362  0.258  0.439831  0.371  0.316  0.446806   
0.5  0.330  0.147  0.543333  0.335  0.279  0.542942  0.346  0.342  0.545697   
0.6  0.315  0.163  0.609206  0.332  0.289  0.648771  0.332  0.368  0.648261   
0.7  0.287  0.179  0.710317  0.314  0.311  0.746798  0.330  0.379  0.745119   
0.8  0.270  0.195  0.844048  0.292  0.347  0.846421  0.325  0.384  0.845180   
0.9  0.256  0.205  0.946667  0.287  0.353  0.870310  0.316  0.400  0.923931   
1.0  0.250  0.211  1.000000  0.287  0.358  1.000000  0.295  0.437  1.000000   

     0.200                   0.400                   0.800                   
        R1     S1        fu     R1     S1        fu     R1     S1        fu  
0.0  0.570  0.337  0.000000  0.645  0.616  0.000000  0.831  0.979  0.000000  
0.1  0.503  0.395  0.149078  0.573  0.700  0.147837  0.811  0.995  0.148745  
0.2  0.490  0.405  0.240030  0.550  0.732  0.248359  0.803  1.000  0.197865  
0.3  0.439  0.453  0.345074  0.502  0.800  0.347770  0.799  1.000  0.250481  
0.4  0.415  0.479  0.449552  0.472  0.847  0.449957    NaN    NaN       NaN  
0.5  0.404  0.495  0.541126  0.469  0.847  0.543147    NaN    NaN       NaN  
0.6  0.373  0.537  0.646655  0.463  0.858  0.564575    NaN    NaN       NaN  
0.7  0.349  0.574  0.749316  0.451  0.884  0.654072    NaN    NaN       NaN  
0.8  0.345  0.579  0.766659  0.430  0.926  0.750459    NaN    NaN       NaN  
0.9  0.331  0.600  0.854041  0.436  0.921  0.851766    NaN    NaN       NaN  
1.0  0.336  0.611  1.000000    NaN    NaN       NaN    NaN    NaN       NaN

In [ ]:
"""Connectome networks relative degree, fd, Fig 7F"""
names = ['SE01','AB01','AB02']
print('name','fd','Fu','R1','S1')
for net,name in zip([se01_undi_net,ab01_undi_net,ab02_undi_net],names):
    fu = round(fractional_asymmetry_undirected(net)[0],3)
    r1,s1 = redundancy_diversity(net,1,directed=False,sex='herm')
    maxedges = (180*179)/2
    fd = round(len(net.edges())/maxedges,3)
    
    print(name,fd,fu,r1,s1)

name fd Fu R1 S1
SE01 0.125 0.356 0.407 0.328
AB01 0.104 0.325 0.431 0.266
AB02 0.101 0.301 0.43 0.252


In [ ]:

''' Pharynx connectoms'''
phadf = pd.read_csv(folder_path+'/raw_data/cne24932-sup-0004-supinfo4.csv',header=[0])
phadfchem = phadf.loc[phadf['Type'] == 'Chemical']
filter = 'BWM|um|anal|int|sph|vm|mc|hyp|pm|intestine|intL|GLRDL|GLRDR|GLRL|GLRR|GLRLR|GLRVL|GLRVR|excgl|sh|g1|g2|bm|e3D|e3VL|e3VR|RIP'
phadfchem = phadfchem[~(phadfchem['Source'].str.contains(filter,regex=True))]
phadfchem = phadfchem[~(phadfchem['Target'].str.contains(filter,regex=True))]
phachemnet = nx.from_pandas_edgelist(phadfchem,source='Source',target='Target')
phadfelec = phadf.loc[phadf['Type'] == 'Electrical']
filter = 'BWM|um|anal|int|sph|vm|mc|hyp|pm|intestine|intL|GLRDL|GLRDR|GLRL|GLRR|GLRLR|GLRVL|GLRVR|excgl|sh|g1|g2|bm|e3D|e3VL|e3VR|RIP'
phadfelec = phadfelec[~(phadfelec['Source'].str.contains(filter,regex=True))]
phadfelec = phadfelec[~(phadfelec['Target'].str.contains(filter,regex=True))]
phaelecnet = nx.from_pandas_edgelist(phadfelec,source='Source',target='Target')

In [ ]:
""" other connectomes, Fig 8"""

def get_network_from_csv(filename,directed=bool):
    df = pd.read_csv(filename,index_col=[0])
    if directed:
        G = nx.from_pandas_adjacency(df,create_using=nx.DiGraph)
        for comp in list(nx.strongly_connected_components(G)):
            if len(comp) <3:
                for node in comp:
                    G.remove_node(node)
    elif not directed:
        G = nx.from_pandas_adjacency(df,create_using=nx.Graph)
        if G is not nx.is_connected(G):
            for comp in list(nx.connected_components(G)):
                if len(comp) <3:
                    for node in comp:
                        G.remove_node(node)
    return(G)

male_chem = get_network_from_csv(folder_path+'/raw_data/male_chemical_connectome_cleaned_c_elegans_no_pharynx.csv',directed=False)
male_elec = get_network_from_csv(folder_path+'/raw_data/male_electrical_connectome_cleaned_c_elegans_no_pharynx.csv',directed=False)
se00_elec = get_network_from_csv(folder_path+'/raw_data/herm_electrical_connectome_cleaned_c_elegans_no_pharynx.csv',directed=False)

def network_info(G,directed=bool,sex=str):
    edges = len(G.edges())
    nodes = len(G.nodes())
    maxedges = (nodes*(nodes-1))/2
    fd = edges/maxedges
    cc = nx.average_clustering(G)
    if not directed:
        fu = fractional_asymmetry_undirected(G)[0]
        r1,s1 = redundancy_diversity(G,1,directed=False,sex=sex)
        fuR1, fpR1, Rp1, fpS1, Sp1, Su1, FuS1 = calculate_fuR1fpS1(G,sex=sex,directed=False)
        sp = nx.average_shortest_path_length(G)

    return(round(fu,3),round(fd,3),round(r1,3),round(Rp1,3),round(s1,3),round(Su1,3),round(fpR1,3),round(FuS1,3))

infodic = {}
netlis = [se00_undi_net,se01_undi_net,ab01_undi_net,ab02_undi_net,se00_elec,male_chem,male_elec,l101_undi_net,l201_undi_net,l301_undi_net,phachemnet]
name = ['CeH0c','CeH1c','CeH2c','CeH3c','CeH0e','CeMc','CeMe','CeL1c','CeL2c','CeL3c','CePc']
for net, name in zip(netlis,name):
    if 'M' in name: 
        Fu,Fd,r1,Rp1,s1,Su1,fpr1,fus1=network_info(net,directed=False,sex='male')
        infodic[name] = {'fu':Fu,'fd':Fd,'R1':r1,'Rp1':Rp1,'S1':s1,'Su1':Su1,'FpR1':fpr1,'FuS1':fus1}
    else:
        Fu,Fd,r1,Rp1,s1,Su1,fpr1,fus1=network_info(net,directed=False,sex='herm')
        infodic[name] = {'fu':Fu,'fd':Fd,'R1':r1,'Rp1':Rp1,'S1':s1,'Su1':Su1,'FpR1':fpr1,'FuS1':fus1}
infodf = pd.DataFrame.from_dict(infodic,orient='index')
infodf

,fu,fd,R1,Rp1,S1,Su1,FpR1,FuS1
CeH0c,0.363,0.074,0.419,0.270,0.272,0.177,0.644,0.651
CeH1c,0.356,0.125,0.407,0.262,0.328,0.214,0.644,0.652
CeH2c,0.325,0.104,0.431,0.286,0.266,0.162,0.664,0.611
CeH3c,0.301,0.101,0.430,0.301,0.252,0.146,0.700,0.578
CeH0e,0.499,0.028,0.339,0.170,0.099,0.075,0.501,0.753
CeMc,0.447,0.050,0.388,0.215,0.147,0.101,0.554,0.686
CeMe,0.532,0.023,0.325,0.149,0.049,0.035,0.458,0.707
CeL1c,0.375,0.048,0.393,0.267,0.136,0.079,0.679,0.582
CeL2c,0.337,0.078,0.415,0.285,0.197,0.117,0.687,0.594
CeL3c,0.271,0.078,0.434,0.321,0.192,0.100,0.740,0.521
